In [ ]:
# 현장에서 많이씀(성능이 좋기때문) => RandomForest

In [ ]:
params 
KNN { 'n_neighbors' : range(3,51,2), 'metric' : ['euclidean', 'manhattan']  }
DecisionTree { 'max_depth':range(2,11), 'min_samples_leaf':range(10,101,10) }
RandomForest { 'n_estimators':[20,50,100], 'max_features':range(1,21) }
XGBoost {'learning_rate' : np.linspace(0.01,0.2, 20), 'n_estimators':range(60,200,20), 'max_depth':[3,4,5,6]}
SVM { 'C' : np.linspace(0.01, 100, 50), 'gamma':[0.001,0.01,.1,1] }

GridSearchCV(m, params, cv=5, scoring = 'neg_mean_absolute_error')
XGBRegressor(objective = 'reg:squarederror')

## 라이브러리

In [ ]:
# 시각화 한글폰트 설정
import seaborn as sns

sns.set(font="NanumGothicCoding", 
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import * 

# 비지도 학습
from sklearn.datasets import make_blobs, make_moons
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, DBSCAN

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings(action='ignore')

## target 변수 조회

In [ ]:
print(data['Creditability'].value_counts())
print(data['Creditability'].value_counts()/ data.shape[0])

data['Creditability'].value_counts().plot(kind = 'barh')
plt.show()

## 저장

In [ ]:
submission['label'] = submission_pred
submission

submission['label']= submission['label'].replace([0,1],['ham','spam'])

submission.to_csv("/aihub/data/M2.csv",index=False)

## to_datetime

In [ ]:
df_total['기준일ID'] = df_total['기준일ID'].astype('str')

air_201['time'] = pd.to_datetime(air_201['time'],format='%Y%m%d%H')
air_202['time'] = pd.to_datetime(air_202['time'],format='%Y%m%d%H')

## 데이터 준비

In [ ]:
# 데이터분할1
target = 'Sales'
x = data.drop(target, axis=1)
y = data.loc[:, target]

# 가변수화
dumm_cols = ['ShelveLoc','Education','Urban', 'US']
x = pd.get_dummies(x, columns = dumm_cols, drop_first = True)

# 데이터 분할2
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2
                                                  , random_state = 2022)

# 스케일링
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

x_train_s = pd.DataFrame(x_train_s, columns=list(x))
x_val_s = pd.DataFrame(x_val_s, columns=list(x))

## CatBoost

In [ ]:
!pip install catboost
!pip install category_encoders

from catboost import CatBoostRegressor, Pool

In [ ]:
# 기본모델
cat_model= CatBoostRegressor()
cat_model.fit(train_x, train_y)
cat_predict = cat_model.predict(test_x)

print(mean_squared_error(test_y, cat_predict, squared=False))
print(r2_score(test_y, cat_predict))

In [ ]:
# 최적화
CB = CatBoostRegressor(depth=4,bagging_temperature=2.099,learning_rate=0.02091,subsample=0.2325)
CB.fit(train_x, train_y)
CB_pred = CB.predict(test_x)

* 튜닝

In [ ]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

def fit_model(train_x,train_y,test_x,depth,bagging_temperature,learning_rate,subsample):
    cat_model = CatBoostRegressor(depth = int(depth),
                                  bagging_temperature=bagging_temperature,
                                  learning_rate=learning_rate,
                                  random_state=1339,#
                                  verbose=0,#
                                  subsample=subsample
                                  ).fit(train_x,train_y)
    cat_predict = cat_model.predict(test_x)
    return cat_predict

def CAT_cv(depth,bagging_temperature,learning_rate,subsample):

    cat_predict = fit_model(np.array(train_x),np.array(train_y),np.array(test_x),depth,bagging_temperature,learning_rate,subsample)
    score = r2_score(test_y,cat_predict)

    return score

In [ ]:
pbounds = { 'depth': (3, 15),
            'bagging_temperature': (1, 10),
            'learning_rate': (0.01, 1),
            'subsample' : (0.01,1),
            }

import numpy as np

bo = BayesianOptimization(f = CAT_cv, pbounds = pbounds, random_state = 42,verbose = 2)
bo.maximize(init_points = 5, n_iter = 10,acq = 'ei',xi = 0.01)

In [ ]:
# ensemble model
from sklearn.ensemble import VotingRegressor

cat1 = CatBoostRegressor(depth=5,bagging_temperature=2.909,learning_rate=0.01,subsample=0.7559)
cat2 = CatBoostRegressor(depth=4,bagging_temperature=2.308,learning_rate=0.01,subsample=1)
cat3 = CatBoostRegressor(depth=4,bagging_temperature=2.099,learning_rate=0.02091,subsample=0.2325)


eclf = VotingRegressor(estimators=[
         ('cat1', cat1), ('cat2', cat2),('cat3', cat3)])

eclf.fit(train_x, train_y)
eclf_predict = eclf.predict(test_x)

In [ ]:
print(np.sqrt(mean_squared_error(test_y,eclf_predict)))
print(r2_score(test_y,eclf_predict))

## Fitting Graph ->Elbow Method [모델과적합]

* 가장 단순한 모델(평균모델)
** knn : k를 최대로 크게하면 평균 모델이 됨.
** k의 최대값은 학습 데이터의 행 수

* Decision Tree
** Decision Tree는 나무의 크기가 클 수록 복잡한 모델
** 크기를 결정하는 파라미터는 max_depth

In [ ]:
# 모델링
n = x_train_s.shape[0]
model = KNeighborsRegressor(n_neighbors = n) # train set의 행 수
model.fit(x_train_s, y_train)
pred_train = model.predict(x_train_s)

In [ ]:
result_train = [] # train set을 가지고 예측한 결과
result_val = [] # val set을 가지고 예측한 결과
k_values = list(range(1,101))

# KNN
for d in k_values :
    model = KNeighborsClassifier(n_neighbors= d)
    model.fit(x_train_s, y_train)
    pred_tr, pred_val = model.predict(x_train_s), model.predict(x_val_s)
    result_train.append(accuracy_score(y_train, pred_tr))
    result_val.append(accuracy_score(y_val, pred_val))
    
result_train = [] # train set을 가지고 예측한 결과
result_val = [] # val set을 가지고 예측한 결과
depth = list(range(1,21))

# Decision Tree
for d in depth :
    model = DecisionTreeClassifier(max_depth = d)
    model.fit(x_train, y_train)
    pred_tr, pred_val = model.predict(x_train), model.predict(x_val)
    result_train.append(accuracy_score(y_train, pred_tr))
    result_val.append(accuracy_score(y_val, pred_val))

In [ ]:
plt.figure(figsize = (12,8))
plt.plot(k_values, result_train, label = 'train_acc', marker = 'o')
plt.plot(k_values, result_val, label = 'val_acc', marker = 'o')

plt.xlabel('Complexity')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

## Decision Tree[ 변수중요도 그래프 ] 실전에서 많이 씀

In [ ]:
# 이 이름을 이용하여 시각화
plt.figure(figsize = (20,8)) # 그림 사이즈 조절
plot_tree(m3.best_estimator_, feature_names = x_train.columns, 
               class_names= ['Bad', 'Good'], filled = True, fontsize = 10);

In [ ]:
def plot_feature_importance(importance, names):
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_df.reset_index(drop=True, inplace = True)

    plt.figure(figsize=(10,8))
    sns.barplot(x='feature_importance', y='feature_names', data = fi_df)

    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    plt.grid()

    return fi_df

In [ ]:
result = plot_feature_importance(model.feature_importances_, list(x_train))

In [ ]:
result = plot_feature_importance(model.best_estimator_.feature_importances_, list(x_train)) # 튜닝했기 때문에

## Regessor 차트 확인

In [ ]:
# 튜닝 과정 로그를 df로 저장 합시다.
result = pd.DataFrame(m1_gs.cv_results_)

# 튜닝 결과를 그래프로 그려봅시다.
plt.figure(figsize = (10,6))
sns.lineplot(x='param_max_depth', y='mean_test_score', data = result)  # plt.plot
plt.grid()
plt.show()

In [ ]:
SVR(Regressor)
# 이를 차트로 그려봅시다.
plt.figure(figsize = (12,8))
sns.lineplot(x = 'param_C', y = 'mean_test_score', data = result, hue = 'param_gamma')
plt.grid()
plt.show()

## XGB 변수중요도 / 시각화

In [ ]:
#위에꺼 plot_feature_importance 함수 가져오기

In [ ]:
# xgb의 변수 중요도
weight : 모델 전체에서 해당 feature가 split될 때 사용된 횟수의 합(plot_tree 에서의 기본값)
gain : feature별 평균 imformation gain.(model.feature_importances_ 의 기본값)
cover : feature가 split 할때의 샘플 수의 평균.

In [ ]:
from xgboost import XGBRegressor, plot_tree, plot_importance
plt.rcParams['figure.figsize'] = 8, 5
plot_importance(model)
plt.show()

In [ ]:
result = plot_feature_importance(model.feature_importances_, list(x),6) #x_train

In [ ]:
# 시각화(Tree하나씩 열어볼 수 있음)
plt.rcParams['figure.figsize'] = 20,20  # 파일전체에 영향을 미침 !!!!
plot_tree(model, num_trees = 0) 
plt.show()

# xgboost 자체 plot_tree 함수를 제공합니다.
# plot_tree(model, num_trees = 0)
# num_trees : 전체 트리 5개짜리 모델이므로 각각 0~4까지 인덱스로 조회해 볼 수 있다.

## 성능비교

In [ ]:
RMSE, MAE, MAPE = [],[],[]
model_desc = ['lr_selected', 'lr_all','knn','dt','rf','xgb','svm']
pred = [p1, p2, p3, p4, p5, p6,p7]

for i, p in enumerate(pred) :
    RMSE.append(mean_squared_error(y_val, p, squared=False))
    MAE.append(mean_absolute_error(y_val, p))
    MAPE.append(mean_absolute_percentage_error(y_val, p))

result = pd.DataFrame({'model_desc':model_desc,'RMSE':RMSE,'MAE':MAE,'MAPE':MAPE})
result

In [ ]:
print(confusion_matrix(y_val, pred1))
print(classification_report(y_val, pred1))

## Decision Tree 시각화 및 변수중요도

In [ ]:
plt.figure(figsize = (10,10)) # 그림 사이즈 조절
plot_tree(m1, feature_names = list(x_train), 
               class_names= ['Stay', 'Leave'], filled = True, fontsize = 10);  # class_names = [target의 내용]
plt.show()
print('-'*88)
# 변수 중요도
print(list(x_train))
print(m1.feature_importances_)
print('-'*88)
print(classification_report(y_val, p1))

## 로지스틱 회귀를 위한 전진선택법 함수 

In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

def forward_stepwise_linear(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    # 
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = OLS(y_train, add_constant(x_tr)).fit(disp=False)
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)
        
        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)
    
    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

In [ ]:
vars, result = forward_stepwise_logistic(x_train, y_train)

In [ ]:
# 선택된 변수
lr_m1 = LinearRegression()
lr_m1.fit(x_train[vars], y_train)
p1 = lr_m1.predict(x_val[vars])

print('RMSE : ', mean_squared_error(y_val, p1, squared=False))
print('MAE  : ', mean_absolute_error(y_val, p1))
print('MAPE : ', mean_absolute_percentage_error(y_val, p1))

## cross_val_score

In [ ]:
from sklearn.model_selection import cross_val_score
# train + validation set을 이용하여 학습, 예측, 평가를 한번에. (여기서는 .fit 이 아님!)
dt_result = cross_val_score(model, x, y, cv=10)
print(dt_result)
print(dt_result.mean(), dt_result.std())

## Permutation Feature Importance(그 외 변수중요도 그래프)

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
pfi1 = permutation_importance(model4, x_val_s, y_val, n_repeats=10, scoring = 'r2', random_state=2022)
# scoring = 'accuracy' default[분류 모델]
# deep learning 모델에 대해서는 명시적으로 scoring = 'r2'을 지정해 필요

In [ ]:
# feature별 Score 분포
plt.figure(figsize = (10,8))
for i,vars in enumerate(list(x)) :
    sns.kdeplot(pfi1.importances[i], label = vars)
plt.grid()
plt.legend()
plt.show()

In [ ]:
sorted_idx = pfi1.importances_mean.argsort()
plt.figure(figsize = (10, 8))
plt.boxplot(pfi1.importances[sorted_idx].T, vert=False, labels=x.columns[sorted_idx])
plt.axvline(0, color = 'r')
plt.grid()
plt.show()

In [ ]:
# 평균값으로 변수중요도 그래프 그리기
pfi1 = permutation_importance(model1, x_val_s, y_val, n_repeats=10, scoring = 'r2', random_state=2022)

In [ ]:
# 변수 중요도 plot(가져오기)
result = plot_feature_importance(pfi1.importances_mean, list(x_train))

## Partial Dependence Plot(개별 변수별 관계)

* 변수 중요도,PDP : train 데이터로 살펴보는 것이 기본이다. 그리고 결과가 유사해야 한다. 

In [ ]:
from sklearn.inspection import plot_partial_dependence, partial_dependence

In [ ]:
# 개별 변수 분석
var = 'MonthlyIncome'

plt.rcParams['figure.figsize'] = 10, 6
plot_partial_dependence(model, features = [var], X = x_val)
plt.grid()
plt.show()

In [ ]:
# 전체 데이터로 확인
var = 'MonthlyIncome'

plt.rcParams['figure.figsize'] = 12, 8
plot_partial_dependence(model, features = [var], X = x_val, kind = 'both') #kind = 'average'
plt.grid()
plt.show()

In [ ]:
# 두개 변수 비교
plot_partial_dependence(model, features = ['rm','lstat'], X = x_train)
plt.show()

In [ ]:
# 두개 변수 비교
plot_partial_dependence(model2, features = [('CreditAmount','Age')], X = x_val)
plt.show()

In [ ]:
# 스케일링한거 데이터 프레임에 넣어서 뽑아야함
x_train_s = pd.DataFrame(x_train_s, columns = list(x))  # 칼럼이름 지정 필요!!
x_val_s = pd.DataFrame(x_val_s, columns = list(x))

## SHAP 값으로 모델의 예측 설명하기

In [ ]:
# 선형회귀는 회귀계수로 변수 기여도 해석해도 무방

In [ ]:
# (회귀모델)
explainer = shap.TreeExplainer(model)
shap_values = explainer1.shap_values(x_train)

# (분류모델)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x)

In [ ]:
# 예측하기 위해서는 입력데이터(x)가 2차원이어야 합니다.
pred = model1.predict(x_train.iloc[0:1,:])
pred

In [ ]:
# train의 평균(회귀)
explainer1.expected_value

# train의 평균(분류)
explainer1.expected_value[1]

In [ ]:
# 인덱스별 데이터
shap.initjs() # javascript 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.-> 아나콘다에서는 한번만 !
index=371
# force_plot(전체평균, shapley_values, input)
shap.force_plot(explainer1.expected_value, shap_values1[index,:], x.iloc[index,:])

# 분류모델
shap.initjs() # javascript 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.-> 아나콘다에서는 한번만 !
index=932
# force_plot(전체평균, shapley_values, input)
shap.force_plot(explainer.expected_value[1], shap_values[1][index,:], x.iloc[index,:])

In [ ]:
# 전체 데이터
shap.initjs() # javascript 시각화 라이브러리 --> colab에서는 모든 셀에 포함시켜야 함.-> 아나콘다에서는 한번만 !

# force_plot(전체평균, shapley_values, input)
shap.force_plot(explainer1.expected_value, shap_values1[0, :], x_train.iloc[0,:])

## class balance를 맞추기 위한 resampling

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

## 비지도 학습

In [ ]:
ks = range(1,50)
inertias = []

for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(mobile_x)
    inertias.append(model.inertia_)

In [ ]:
plt.figure(figsize=(20,8))
# Plot ks vs inertias
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
# 모델링[KNN]
model = KMeans(n_clusters=5)
model.fit(mobile_x)
pred = model.predict(mobile_x)
pred = pd.DataFrame(pred, columns = ['predict'])
# 결과 보기
mobile_y.reset_index(inplace=True, drop=True)
result = pd.concat([mobile_x, mobile_y, pred], axis =1)
result.CHURN = result.CHURN.astype('int')
# 클러스터 별 고객이탈율
result.groupby('predict')['CHURN'].mean()
# 전체 평균
result['CHURN'].value_counts() / result.shape[0]

In [ ]:
# DBSCAN 모델을 만들어 봅시다.
model = DBSCAN(eps=0.1, min_samples=3)
model.fit(x)
# fitting한 후에 모델의 labels_ 값이 찾아낸 군집 종류입니다.
clusters = model.labels_
# 군집 번호 중 -1은 이상치를 의미합니다.(어느 군집에도 포함 안되는 값들!)
clusters

## 시계열 데이터

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore')
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
# target 시계열로 확인 
plt.figure(figsize = (20,8))
plt.plot(data['sales'])
plt.grid()
plt.show()

temp = data[-100:]
plt.figure(figsize = (20,8))
plt.plot(temp['sales'], marker ='o')
plt.grid()
plt.show()

In [ ]:
# 잔차 분석
def residual_diag(residuals, lags = 20) :

    print('* 정규성 검정(> 0.05) : ', round(spst.shapiro(residuals)[1],5))
    print('* 정상성 검정(< 0.05) : ', round(sm.tsa.stattools.adfuller(residuals)[1],5))
    print('* 자기상관성 확인(ACF, PACF)')
    fig,ax = plt.subplots(1,2, figsize = (15,5))
    plot_acf(residuals, lags = lags, ax = ax[0])
    plot_pacf(residuals, lags = lags, ax = ax[1])
    plt.show()

In [ ]:
# 날짜 타입으로 변경
data['date'] = pd.to_datetime(data['date'])
# 날짜를 인덱스로 변환
data['DT'] = data['date']
data.set_index('DT', inplace=True)
data.head()
## 날짜단위 지정하기 : freq / 인덱스 조회시, 마지막에 있는 freq 옵션
# 일단위
data.asfreq('D').head()
# 월(말)단위
data.asfreq('M').head()
# 월초 단위
data.asfreq('MS').head()
#(추가) 빠진값 찾기
temp = data.asfreq('D')
temp.isna().sum()
# 채우기
data.asfreq('D', method = 'ffill')
df = data.asfreq('D') # 일단위 데이터이므로 이걸로 지정

In [ ]:
# y 만들기
df['y'] = df['sales'].shift(-1)
display(df.head())
display(df.tail())
# 제일 마지막 행은 삭제
df.dropna(axis = 0, inplace = True)
df.tail()

In [ ]:
# 데이터 분할
# 1) x, y 나누기
# .values(넘파이 어레이)로 변환해서 저장하는 이유 ➡ 데이터 스플릿 index를 적용해서 데이터를 가져오기 위해서
target = 'y'

x = df.drop([target, 'date'], axis = 1)
y = df.loc[:, target]

# 시계열 데이터 분할
from sklearn.model_selection import TimeSeriesSplit
x.shape
# validation set size
val_size = 30
nfold = 3

tscv = TimeSeriesSplit(n_splits = nfold, test_size = val_size)
tscv

#참조
# .split을 이용하여 fold 하나씩 인덱스들을 뽑아 낼 수 있음.
for train_index, val_index in tscv.split(x):
    print("Train:", train_index, "Val:", val_index)

In [ ]:
# 모델링
# loop 돌며 모델링(cross-validation) 수행
rmse, mae, mape = [],[],[]
residuals = []
pred = []
model = LinearRegression()

for train_index, val_index in tscv.split(x):

    # 인덱스로 데이터 분할
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_val, y_val = x.iloc[val_index], y.iloc[val_index]

    # 학습
    model.fit(x_train, y_train)

    # 예측
    pr = model.predict(x_val)
    pred += list(pr)

    # 평가
    rmse.append(mean_squared_error(y_val, pr, squared = False))
    mae.append(mean_absolute_error(y_val, pr))
    mape.append(mean_absolute_percentage_error(y_val, pr))

    # 잔차 : 각 fold의 결과를 리스트로 변환하여 추가
    residuals += list(y_val - pr)

In [ ]:
# 예측 결과 평가
print('RMSE : ',round(np.mean(rmse),4))
print('MAE  : ',round(np.mean(mae),4))
print('MAPE : ',round(np.mean(mape),4))

In [ ]:
# 그래프 비교
n = val_size * nfold
pred = pd.Series(pred, index = y[-n:].index)
plt.figure(figsize = (20,8))
plt.plot(y[:-n], label = 'train')
plt.plot(y[-n:], label = 'val')
plt.plot(pred, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

plt.figure(figsize = (20,8))
plt.plot(y[-n:], label = 'val')
plt.plot(pred, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

In [ ]:
# 평가: 잔차분석
## 시각화
plt.figure(figsize = (12,8))
plt.plot(residuals)
plt.axhline(0, color = 'r', ls = '--')
plt.axhline(np.mean(residuals), color = 'g', ls = '--')
plt.show()

## ACF, PACF(자기상관성 여부 확인)
lags = 20

fig,ax = plt.subplots(1,2, figsize = (15,5))
plot_acf(residuals, lags = lags, ax = ax[0])
plot_pacf(residuals, lags = lags, ax = ax[1])
plt.show()

## 검정
from scipy import stats
import statsmodels.api as sm
stats.shapiro(residuals)[1]  # 정규성 검정 : Shapiro-Wilk 검정
sm.tsa.stattools.adfuller(residuals)[1]  # 정상성 검정 : ADF 검정

## 시계열 모델링

In [ ]:
# 결과 시각화
def plot_model_result(y_train, y_val, pred) :
    pred = pd.Series(pred, index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

In [ ]:
# y값 살펴보기
residual_diag(y_train, lags = 30)

# ARIMA 모델링
m1_1 = sm.tsa.SARIMAX(y_train, order=(1,0,1)).fit() # ARMA
m1_2 = sm.tsa.SARIMAX(y_train, order=(1,1,1)).fit() # ARIMA

# SARIMA 모델링 : P, D, Q, m = 1,1,1,7 로 모델을 생성합시다.
m2_1 = sm.tsa.SARIMAX(y_train, order=(5,1,4), seasonal_order=(1,1,1,7)).fit()

m3_1 = sm.tsa.SARIMAX(y_train, order=(5,1,4), seasonal_order=(1,1,1,7), exog=x_train).fit()

# 평가[잔차 진단]
residuals = m1_1.resid  # y_train과 예측값 차이
residual_diag(residuals)
# 평가[AIC] ->선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
# 값이 작을 수록 좋은 모델
# 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수
print('model1 AIC :', m1_1.aic)
# 평가[Validation]
pred = m1_1.forecast(30)   # SARIMAX 모델을 생성하고, 예측할 때는 exog=x_val 옵션이 들어가야 함. 
print('MAE :', mean_absolute_error(y_val, pred))
print('MAPE:', mean_absolute_percentage_error(y_val, pred))
# 평가[결과 시각화]
plot_model_result(y_train, y_val, pred)

In [ ]:
# 하이퍼파라미터 튜닝

# 학습
from itertools import product
# product 함수를 이용하여 값의 조합을 구성
p = [1,2,3,4,5]
q = [1,2,3,4,5]
d = [1]
iter = list(product(p,d,q))
iter

# 튜닝 
mae, aic = [],[]
for i in iter :
    model_fit = sm.tsa.SARIMAX(y_train, order=(i[0],i[1],i[2])).fit()
    pred = model_fit.forecast(30)
    mae.append( mean_absolute_error(y_val, pred))
    aic.append(model_fit.aic)
    print(i)
    
result = pd.DataFrame({'params(p,d,q)' : iter, 'mae' : mae, 'aic':aic})

display(result.loc[result['mae'] == result.mae.min()])
display(result.loc[result['aic'] == result.aic.min()])

# 가장 성능이 좋은 p, d, q 값으로 모델을 생성합니다.
m1_3 = sm.tsa.SARIMAX(y_train, order=(5,1,4)).fit()

# 평가[잔차진단]
residuals = m1_3.resid
residual_diag(residuals) # seasonallity 
# 평가[AIC]
print('model2 AIC :', m1_3.aic)
# 평가[validation]
p1 = m1_3.forecast(30)
print('MAE :', mean_absolute_error(y_val, p1))
print('MAPE:', mean_absolute_percentage_error(y_val, p1))
# 결과 시각화
plot_model_result(y_train, y_val, p1)

## 딥러닝[keras]

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [ ]:
x.shape , y.shape  # 확인해볼것

* Functional

In [ ]:
@ [Linear Regression]
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
keras.backend.clear_session()

# 레이어들을 사슬로 연결하 듯이 연결!
input_layer = keras.layers.Input(shape=(1,))
output_layer = keras.layers.Dense(1)(input_layer)

# 모델의 시작과 끝을 지정
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss = 'mse', optimizer = 'adam')

model.fit(x, y, epochs=10, verbose=1)
# 결과 출력
print(model.predict(x).reshape(-1,) )

@ [Logistic Regression]
keras.backend.clear_session()
input_layer = keras.layers.Input(shape=(1,))
output_layer = keras.layers.Dense(1, activation='sigmoid')(input_layer)
model = keras.models.Model(inputs=input_layer, outputs=output_layer)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.fit(x, y, epochs=10, verbose=1)
print(y)
print(model.predict(x).reshape(-1,) )

* Sequential

In [ ]:
@ [Linear Regression]
# 1번 청소 : 이미 만들어진 모델이 있다면 그 모델을 없애줘
keras.backend.clear_session()

# 2번 모델 선언
model = keras.models.Sequential()

# 3번 모델 블록 조립
model.add( keras.layers.Input(shape=(1,)) )
model.add( keras.layers.Dense(1) )

## 오리지널 Sequential API
# model.add( keras.layers.Dense(1, input_shape=(1,)) )

# 4번 컴파일 
model.compile(loss='mse',optimizer='adam')
model.fit(x[:15], y[:15], epochs=10, verbose=1)
print(y[15:])
print(model.predict(x[15:]))

@ [Logistic Regression]
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
keras.backend.clear_session()

# model에 순차적으로 레이어를 쌓아가겠다는 의도!
model = keras.models.Sequential()

# model에 인풋 값을 받는 레이어를 넣음
model.add( keras.layers.Input(shape=(1,)) )
# model에 Dense 레이어를 넣을거야 (최초의 레이어) : weight를 곱하고, bias를 더해주는 과정
model.add( keras.layers.Dense(1, activation='sigmoid') )

# 오리지널 Sequential API
# model.add( keras.layers.Dense(1, input_shape=(1,), activation='sigmoid') )

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics=['accuracy'])
# keras.losses.binary_crossentropy 이걸로도 가능

model.fit(x[:15], y[:15], epochs=10, verbose=1)
print(y[15:])
print(model.predict(x[15:]))

* 멀티클래스

In [ ]:
# 미래 데이터를 먼제 떼어내야하기 때문에 먼저 분리해준다.(미래데이터는 건드리지 않는다)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.2, random_state=2022)
x_train.shape, y_train.shape
# One-Hot Encoding  (get_dummies= x에만 적용)
class_n = len(np.unique(y_train))
y_train = to_categorical(y_train, class_n)
y_test = to_categorical(y_test, class_n)

In [ ]:
@ y 확인하기
iris.target_names

# One-Hot Encoding  (get_dummies= x에만 적용)
from tensorflow.keras.utils import to_categorical 

y = to_categorical(y, 3) # 반복 실행 주의!!(계속 생성됨)
x.shape, y.shape

@ [Sequential]
keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(4,)))
model.add(keras.layers.Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,verbose=1)
model.predict(x_test).reshape(-1)
y.argmax(axis=1)

@ [Functional]
keras.backend.clear_session()
il = keras.layers.Input(shape=(4,))
ol = keras.layers.Dense(3,activation='softmax')(il)
model = keras.models.Model(il,ol)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
model.fit(x_train,y_train,epochs=10, verbose=1)
pred = model.predict(x_test)
pred[:5] # 합치면 확률값 =1

* Hidden layer 추가

In [ ]:
@ [Sequential API]

keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(13,)))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(1))
model.compile(loss='mse',optimizer='adam')
model.summary()

@ [Functional]
keras.backend.clear_session()
il = keras.layers.Input(shape=(30,))
hl = keras.layers.Dense(32, activation='relu', name='hidden1')(il)
hl = keras.layers.Dense(32, activation='relu', name='hidden2')(hl)
ol = keras.layers.Dense(1,activation='sigmoid')(hl)
model = keras.models.Model(il,ol)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

@ [multi-Functional]
keras.backend.clear_session()
il = keras.layers.Input(shape=(4,))
hl = keras.layers.Dense(32, activation='relu', name='hidden1')(il)
hl = keras.layers.Dense(32, activation='relu', name='hidden2')(hl)  # 변수명 같아도 상관없음
ol = keras.layers.Dense(3,activation='softmax')(hl)
model = keras.models.Model(il,ol)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # optimizer=keras.optimizers.Adam(0.01)-> eta
model.summary()

## 딥러닝[ANN]

In [ ]:
# 참조코드
'''
matplolib inline 명령어를 통해서
matplot으로 그리는 플롯들을 주피터 노트북 내에서 볼 수 있게 해준다.
포맷을 retina로 바꾸면 그래프의 화질이 훨씬 좋아진다.
'''
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.activations import relu, softmax

from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [ ]:
1) 전처리
x = wine.data
y = wine.target
x.shape, y.shape
data.target_names

## reshape -> flatten하면 필요없음
train_x.shape
train_x = train_x.reshape([train_x.shape[0],-1])
test_x = test_x.reshape([test_x.shape[0],-1])
train_x.shape # 28*28
## min-max scaling
max_n, min_n = train_x.max(), train_x.min()
max_n, min_n
train_x = (train_x - min_n) / (max_n - min_n)
test_x = (test_x - min_n) / (max_n - min_n)
print(f'max : {train_x.max()} / min : {train_x.min()}')
## target feature : One-hot Encoding
from tensorflow.keras.utils import to_categorical
len_y = len(set(train_y))
train_y = to_categorical(train_y, len_y)
test_y = to_categorical(test_y, len_y)

2) 모델링
train_x.shape, train_y.shape
keras.backend.clear_session()
model = keras.models.Sequential()
model.add(keras.layers.Input(train_x.shape[1]))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.01),metrics='accuracy')
model.summary()

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',  # 무얼 관찰할지(관측대상)
                   min_delta=0,         # 최소한 나빠지지 않으면 괜찮아
                   patience=5,          # 중요! 몇번이나 참을지(거기까지 개선 안되면 멈출거야)
                   verbose=1,            
                   restore_best_weights=True)  # (반드시 사용)학습이 멈췄을 때, 최적의 가중치로 전환해줌
model.fit(train_x, train_y, 
          validation_split=0.2,  # Train data의 20%를 Validation data로!
          callbacks=[es],        # Early Stopping 적용
          verbose=1, epochs=50)

3) 예측
pred_train = model.predict(train_x)
pred_test = model.predict(test_x)
single_pred_train = pred_train.argmax(axis=1)
single_pred_test = pred_test.argmax(axis=1)
logi_train_accuracy = accuracy_score(train_y.argmax(axis=1), single_pred_train)
logi_test_accuracy = accuracy_score(test_y.argmax(axis=1), single_pred_test)
print('트레이닝 정확도 : {:.2f}%'.format(logi_train_accuracy*100))
print('테스트 정확도 : {:.2f}%'.format(logi_test_accuracy*100))

4) 확인
mnist_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
figure = plt.figure(figsize=(20, 10))
for i, index in enumerate(np.random.choice(test_x.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(test_x[index].reshape([28,-1])), cmap='gray' )
    
    predict_index = pred_test[index].argmax(axis=0)
    true_index = test_y[index].argmax(axis=0)
    # Set the title for each image
    ax.set_title(f"{mnist_labels[predict_index]} ({mnist_labels[true_index]})",
                 color=("green" if predict_index == true_index else "red"))
    
5) 오답확인
true_false = (test_y.argmax(axis=1) == single_pred_test)
f_id = np.where(true_false == False)[0]
f_n = len(f_id)
id = f_id[rd.randrange(0,f_n)]
print(f'id = {id}' )
print(f'다음 그림은 숫자 {test_y.argmax(axis=1)[id]} 입니다.')
print(f'모델의 예측 : {single_pred_test[id]}')
print(f'모델의 카테고리별 확률 : {np.floor(pred_test[id]*100)}')
if test_y.argmax(axis=1)[id] == single_pred_test[id] :
    print('===============')
    print('정답입니다')
    print('===============')
else : 
    print('===============')
    print('틀렸어요')
    print('===============')
plt.imshow(test_x[id].reshape([28,-1]), cmap='gray')
plt.show()

6) 평가
model.evaluate(test_x, test_y)

## Modeling : multi-input & Concatenate layer

In [ ]:
1) 데이터 불러오기
iris = load_iris()
x = iris.data
y = iris.target
print(data.DESCR) 
df_x = pd.DataFrame(x, columns=iris.feature_names)
# null값 확인 필요
iris.target_names

2) train set, test set 구분하기
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_x, y, test_size=0.1, random_state=2022)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

추가) Scaling (데이터 수치->너무 편차가 커서(min_max 필요))
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
train_x = mm_scaler.fit_transform(train_x)
test_x = mm_scaler.transform(test_x)
pd.DataFrame(train_x, columns=iris.feature_names).describe()
train_x = pd.DataFrame(train_x, columns=iris.feature_names)
test_x = pd.DataFrame(test_x, columns=iris.feature_names)

3) length끼리, width끼리
print(df_x.columns)
tr_x_l = train_x.loc[:, ['sepal length (cm)', 'petal length (cm)'] ]
tr_x_w = train_x.loc[:, ['sepal width (cm)', 'petal width (cm)'] ]
tr_x_l.shape, tr_x_w.shape
te_x_l = test_x.loc[:, ['sepal length (cm)', 'petal length (cm)'] ]
te_x_w = test_x.loc[:, ['sepal width (cm)', 'petal width (cm)'] ]

4) One-hot Encoding
from tensorflow.keras.utils import to_categorical
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)
train_y.shape

5-1) Modeling : multi-input & Concatenate layer
@ Functional API만 가능!!!!
# 1. 세션 클리어
clear_session()
# 2. 레이어 사슬처럼 엮기 : input 2개!
il_l = Input( shape=(2,) )
hl_l = Dense(2, activation=relu)(il_l)
il_w = Input( shape=(2,) )
hl_w = Dense(2, activation=relu)(il_w)
cl = Concatenate()([hl_l, hl_w])
ol = Dense(3, activation=softmax)(cl)
# 3. 모델 시작과 끝 지정
model = Model([il_l, il_w], ol)
# 4. 모델 컴파일
model.compile(loss=categorical_crossentropy, metrics=['accuracy'],
              optimizer=Adam())
model.summary()

5-2) Modeling : multi-input & Add layer
tr_x_p.shape, train_y.shape
keras.backend.clear_session()
il_s = keras.layers.Input(shape=(2,))
hl_s = keras.layers.Dense(6,activation='swish')(il_s)
il_p = keras.layers.Input(shape=(2,))
hl_p = keras.layers.Dense(6,activation='swish')(il_p)
add_l = keras.layers.Add()([hl_s, hl_p])
ol = keras.layers.Dense(3,activation='softmax')(add_l)
model = keras.models.Model([il_s,il_p],ol)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

6) 모델 시각화
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)

7) 학습
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)
model.fit([tr_x_l, tr_x_w], train_y, validation_split=0.1, epochs=1000, verbose=1, callbacks=[es])
        # 모델에 붓는 순서 지켜야됨

## 시각지능 딥러닝

### CNN

In [ ]:
data = io.loadmat("notMNIST_small.mat")

# transform data
X = data['images']
y = data['labels']
resolution = 28
classes = 10

X = np.transpose(X, (2, 0, 1))

y = y.astype('int32')
X = X.astype('float32') #/ 255.

# shape: (sample, x, y, channel)
X = X.reshape((-1, resolution, resolution, 1))

# looking at data; some fonts are strange
i = np.random.randint(0, 18723)
print(i)
plt.imshow( X[i,:,:,0] )
plt.title( "ABCDEFGHIJ"[y[i]] )

# random letters
rows = 6
fig, axs = plt.subplots(rows, classes, figsize=(classes, rows))

for letter_id in range(10):
    letters = X[y == letter_id]
    for i in range(rows):
        ax = axs[i, letter_id]
        ax.imshow(letters[np.random.randint(len(letters)),:,:,0],
                  cmap='Greys', interpolation='none')
        ax.axis('off')
        
# splitting data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

x_train.shape, y_train.shape

max_n, min_n = x_train.max(),x_train.min()

x_train = (x_train - min_n)/ (max_n - min_n)
x_test = (x_test - min_n)/ (max_n - min_n)

len(np.unique(y_train))
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test,10)
x_train.shape, y_train.shape

In [ ]:
keras.backend.clear_session()

il = keras.layers.Input(shape=(28,28,1,))
hl = keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu',)(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu')(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))(hl)
hl = keras.layers.Dropout(.25)(hl)

hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu')(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu')(hl)
hl = keras.layers.BatchNormalization()(hl)

hl = keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))(hl)
hl = keras.layers.Dropout(.25)(ml)

hl = keras.layers.Flatten()(hl)
hl = keras.layers.Dense(512,)(hl)
hl = keras.layers.BatchNormalization()(hl) 
ol = keras.layers.Dense(10, activation='softmax')(hl)

model = keras.models.Model(il,ol)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
keras.backend.clear_session()

il = keras.layers.Input(shape=(28,28,1))
hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same')(il)
hl = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same')(hl)
hl = keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)
hl = keras.layers.Dropout(.25)(hl)

hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same')(hl)
hl = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same')(hl)
hl = keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)
hl = keras.layers.Dropout(.25)(hl)

hl = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same')(hl)
hl = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same')(hl)
hl = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same')(hl)
hl = keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))(hl)
hl = keras.layers.BatchNormalization()(hl)
hl = keras.layers.Dropout(.25)(hl)

hl = keras.layers.Flatten()(hl)
hl = keras.layers.Dense(1024, activation='relu')(hl)
hl = keras.layers.Dense(1024, activation='relu')(hl)
hl = keras.layers.Dense(1024, activation='relu')(hl)
ol = keras.layers.Dense(10, activation='softmax')(hl)

model = keras.models.Model(il ,ol)
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.01), metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True)
model.fit(x_train, y_train,, verbose=1, validation_split=.2, batch_size=1024, epochs=100, callbacks=[es])

In [ ]:
y_pred = model.predict(x_test)

pred_array = np.zeros(shape=(y_pred.shape[0], y_pred.shape[1]))
idx = 0

for arr_val in y_pred :
    # print(arr_val)
    pred_array[idx][arr_val.argmax()] = 1
    idx += 1
    
pred_array.shape

@ 성능평가
from sklearn.metrics import accuracy_score
print( f'{accuracy_score(y_test, pred_array):.4f}' )

@ 문자 이미지 시각화
import random as rd
character = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J'}

rand_n = rd.randrange(0, 3744)

print(f'id = {rand_n}')
print(f'실제 문자 : {character[y_test[rand_n].argmax()]}')
print(f'모델의 문자 예측 : {character[y_pred[rand_n].argmax()]}' )
print(f'모델의 문자별 예측 확률 : {np.round(y_pred[rand_n]*100)}')
# print(f'모델의 문자들 총 확률 : {sum(np.round(y_pred[rand_n]*100))}')

print('====================================================')

if y_test[rand_n].argmax() == y_pred[rand_n].argmax() :
    print('정답')
else :
    print('오답')

print('====================================================')

plt.figure(figsize=(5,5))
plt.imshow(x_test[rand_n].reshape(28, 28), cmap='gray')
plt.title("ABCDEFGHIJ"[y_test[rand_n].argmax()] )
plt.show()

# 틀린 문자만 확인
t_f = ( y_test.argmax(axis=1) == y_pred.argmax(axis=1) )
false_id = np.where(t_f==False)[0]
false_n = len(false_id)

id = false_id[rd.randrange(0, false_n)]

print(f'id = {id}')
print(f'실제 문자 : {character[y_test[id].argmax()]}')
print(f'모델의 문자 예측 : {character[y_pred[id].argmax()]}' )
print(f'모델의 문자별 예측 확률 : {np.round(y_pred[id]*100)}')
# print(f'모델의 문자들 총 확률 : {sum(np.round(y_pred[rand_n]*100))}')

print('====================================================')

if y_test[id].argmax() == y_pred[id].argmax() :
    print('정답')
else :
    print('오답')

print('====================================================')

plt.figure(figsize=(5,5))
plt.imshow(x_test[id].reshape(28, 28), cmap='gray')
plt.title("ABCDEFGHIJ"[y_pred[id].argmax()] )
plt.show()

### 동영상CNN

In [ ]:
# 필요 라이브러리 불러오기.
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

1) 코랩 사용 시 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
video = cv2.VideoCapture(TUTORIAL_PATH + "/tutorial.mp4")
video.isOpened()

2. 경로 확인하기

In [ ]:
# ROOT_PATH 확인 
import os

# 구글 드라이브 내 프로젝트 압축해제된 영역 (구글 드라이브 최상위에 압축해제 시 그대로 실행 수정 X)
WORK_SPACE = ""

if os.getcwd() == '/content' :
  # 구글 드라이브 사용 시 
  ROOT_PATH = "/content/drive/MyDrive/"+WORK_SPACE+"/AIVLE3rd_individual"
else :
  ROOT_PATH = os.path.abspath('..')

# 모델 예측을 위한 test 데이터가 저장되는 경로
TEST_PATH = ROOT_PATH + "/test"
# 테스트 영상 test.mp4 경로 

TEST_VIDEO = TEST_PATH + "/test.mp4"

# 모델 예측을 위해 테스트 영상을 프레임 이미지로 자르고 저장하는 경로 (testGenerator 생성 시 PATH )   
TEST_IMAGE = TEST_PATH + "/image"

# 모델(.h5) 파일이 저장된 경로 (본인이 생성한 모델)
MODEL_PATH = ROOT_PATH + "/model"

3) test data 다운로드

In [ ]:
!pip install gdown

import gdown
import shutil
import zipfile
 
google_path = 'https://drive.google.com/uc?id='
file_id = "10iKCHLPx-YFgkJcDqCgJPl83ZKYHvXmr"
output_name = 'test.zip'

# 파일 다운로드
gdown.download(google_path+file_id,output_name,quiet=False)

# 파일 위치 이동
shutil.move("./" + output_name, ROOT_PATH)

zip_file = ROOT_PATH + "/" + output_name

# 압축해제 
with zipfile.ZipFile(zip_file) as z:
    z.extractall(ROOT_PATH)

os.remove(zip_file)

In [ ]:
# GPU 환경 확인하기
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
video_width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
video_height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
video_length = video.get(cv2.CAP_PROP_FRAME_COUNT)
video_fps = video.get(cv2.CAP_PROP_FPS)

print("가로 :", video_width)
print("세로 :", video_height)
print("총 프레임 수 :" , video_length)
print("FPS :", video_fps)
print("영상 길이 : %d 초 :"  %round(video_length/video_fps))

video.release()

In [ ]:
# 참조 
for x_data, t_data in train_generator:

    print(x_data.shape) 

* 이미지 만들기(CROP)

In [ ]:
# 실습해보세요.
test_IMAGE = "/content/drive/MyDrive/AIVLE3rd_individual/train/samang"

TIME_MEASUERMENT_UNIT = 1 #TIME MEASUREMENT UNIT을 통해 몇 초 단위로 이미지를 저장할 지 선택

if not os.path.exists(test_IMAGE):
  os.mkdir(test_IMAGE)

video = cv2.VideoCapture("/content/drive/MyDrive/AIVLE3rd_individual/video/train/220206_samang.mp4")

while video.isOpened():
  ret,frame = video.read()
  if ret:
    # 현재 프레임 위치 (msec) 
    frame_sec = video.get(cv2.CAP_PROP_POS_MSEC)/1000
    print(frame_sec%1)
    # if (frame_sec % TIME_MEASUERMENT_UNIT == 0.6796875):
    #  filename = test_IMAGE + "/" + str(round(frame_sec)) + ".jpg"
    #  cv2.imwrite(filename, frame) 
  else:
    break

video.release()

In [ ]:
test_IMAGE = "/content/drive/MyDrive/AIVLE3rd_individual/train/boss"

# TIME_MEASUERMENT_UNIT = 1 #TIME MEASUREMENT UNIT을 통해 몇 초 단위로 이미지를 저장할 지 선택

if not os.path.exists(test_IMAGE):
  os.mkdir(test_IMAGE)

video = cv2.VideoCapture("/content/drive/MyDrive/AIVLE3rd_individual/train/06boss")

count = 0

while video.isOpened():
    ret, frame = video.read()

    if ret:
        if(int(video.get(1)) % video.get(cv2.CAP_PROP_FPS) == 0):
            count += 1
            # filename = test_IMAGE + "/" + str(count) + ".jpg"
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백 변환
            cv2.imwrite(filename, frame[55:80, 35:165]) # 코너 로고만 크롭해서 저장
            # cv2.imwrite(filename, frame)
    else: break

In [ ]:
test_IMAGE = "/content/drive/MyDrive/AIVLE3rd_individual/train/20samang"

TIME_MEASUERMENT_UNIT = 1 #TIME MEASUREMENT UNIT을 통해 몇 초 단위로 이미지를 저장할 지 선택

if not os.path.exists(test_IMAGE):
  os.mkdir(test_IMAGE)

video = cv2.VideoCapture("/content/drive/MyDrive/AIVLE3rd_individual/video/train/220220_samang.mp4")

count = 0

while video.isOpened():
    ret, frame = video.read()

    if ret:
        if(round(int(video.get(1)) % video.get(cv2.CAP_PROP_FPS)) == 0):
            count += 1
            filename = test_IMAGE + "/" + str(count) + ".jpg"
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백 변환
            # cv2.imwrite(filename, frame[55:80, 35:165]) # 코너 로고만 크롭해서 저장
            cv2.imwrite(filename, frame)
    else: break


### CROP

* ①'매뉴얼하게 일일이 확인하는 방법'

In [ ]:
# 실습해보세요.
import matplotlib.pyplot as plt
%matplotlib inline

class_map = {
    0: 'jung',
    1: 'park',    
    2: 'sam',
    3: 'sayuri',
}

image_class = glob.glob(IMAGE_PATH + "*/*")
image_class.sort()

fig, axes = plt.subplots(4, 5,figsize=(16, 9))
class_count = 0 
for c in image_class:  
  for i in range(5):
    original_image = cv2.imread(c + "/" + str(i*200) + ".jpg")
    rgb_image = cv2.cvtColor(original_image,cv2.COLOR_BGR2RGB)
    # 특정 영역만 잘라서 이미지 비교
    cropped_image = rgb_image[0:96, 0:320]
    axes[class_count, i].imshow(cropped_image)
    axes[class_count, i].set_title(class_map[class_count] , fontsize=15)    
    axes[class_count, i].axis('off')
  class_count += 1
plt.show()

* ②'이미지 속 동일한 규칙을 찾아내는 방법'

In [ ]:
# [Tip] 
# 이미지 전체를 학습할 필요가 없다. 
# 코너를 구분하는데 필요한 영역은 좌측 상단에 고정되어 있음으로 해당 영역만 잘라내서 학습 데이터를 생성하는 것도 가능

def croppedImage(IMAGE_PATH):
  #이미지를 잘라낼 영역 
  bounding_box = {
      'start_x' : 0, 
      'start_y' : 0,
      'end_x' : 320,    
      'end_y' : 96
  }
  from tqdm.auto import tqdm, trange
  image_list = glob.glob(IMAGE_PATH + '/*/*.jpg')
  for image in tqdm(image_list, desc="IMAGE CROP PROGRESS"):
    original_image = cv2.imread(image)
    cropped_image = original_image[bounding_box['start_y']:bounding_box['end_y'], bounding_box['start_x']:bounding_box['end_x']]    
    cv2.imwrite(image, cropped_image) 

In [ ]:
croppedImage(IMAGE_PATH)

In [ ]:
TRAIN_PATH = ROOT_PATH + "/train"

In [ ]:
import shutil

shutil.copytree(IMAGE_PATH, TRAIN_PATH)

In [ ]:
# 만약 logo 부분만 crop하신다면 굳이 flip을 이용해서 데이터를 증강할 필요는 없습니다

4) 전처리

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32
img_height = 96
img_width = 320

train_datagen = ImageDataGenerator(
    rescale=1. /255,
    validation_split=0.2,
    # rotation_range=40,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # fill_mode='nearest'
)
# train_genrator 생성
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    batch_size=16,
    target_size=(img_height, img_width),
    class_mode='categorical',
    subset='training',
   )

# validation_generator 생성 (train와 동일하게 해야함)
validation_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    batch_size= batch_size,
    target_size=(img_height, img_width),
    class_mode='categorical',
    subset='validation',
    )

In [ ]:
def my_preprocessing(TEST_VIDEO):
  from tensorflow import keras
  from keras.preprocessing.image import ImageDataGenerator
  import os
  import cv2

  TEST_IMAGE = TEST_PATH + "/image"

  TIME_MEASUERMENT_UNIT =1 #TIME MEASUREMENT UNIT을 통해 몇 초 단위로 이미지를 저장할 지 선택

  if not os.path.exists(TEST_IMAGE):
    os.mkdir(TEST_IMAGE)

  video = cv2.VideoCapture(TEST_PATH + "/test.mp4")

  while video.isOpened():
    ret,frame = video.read()
    if ret:
      # 현재 프레임 위치 (msec) 
      frame_sec = video.get(cv2.CAP_PROP_POS_MSEC)/1000
      if frame_sec.is_integer():
        if (frame_sec % TIME_MEASUERMENT_UNIT == 0):
          filename = TEST_IMAGE + "/" + str(round(frame_sec)) + ".jpg"
          cv2.imwrite(filename, frame) 
    else:
      break

  video.release()

  batch_size=8
  img_height=240
  img_width=427

  test_datagen = ImageDataGenerator(
    rescale=1. /255,)
  
# test_genrator 생성
  test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    batch_size=batch_size,
    target_size=(img_height, img_width),
    class_mode='categorical',
    subset='test',
    )      

  return test_generator

In [ ]:
# 결과 확인 (수정하지 마세요.) - Found 1220 images belonging to 1 classes. 와 같은 메시지가 출력되어야 합니다. 
test_generator = my_preprocessing(TEST_VIDEO)

5) 모델 만들기

In [ ]:
# shape찍어보기
from PIL import Image

imagel = Image.open('/content/drive/MyDrive/AIVLE3rd_individual/train/park/1000.jpg')
imagel.show()
imagl_size = imagel.size
print(imagl_size)

In [ ]:
# Conv2D, MaxPooling2D 조합으로 층을 쌓습니다. 첫번째 입력층의 input_shape은 imageDataGenerator target size로 지정합니다.
keras.backend.clear_session()

model = Sequential()

model.add(Input(shape=(img_height, img_width,3,)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu',))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(.25))

model.add(Flatten())
model.add(Dense(100,))
model.add(BatchNormalization())
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[img_height, img_width,3,]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=8, activation='softmax'))

# Compiling the CNN
cnn.compile(optimizer = 'adam', 
            loss = 'categorical_crossentropy', 
            metrics = ['accuracy'])
cnn.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
MODEL_PATH = '/content/drive/MyDrive/AIVLE3rd_individual/weights'
# early_stopping 
cp = ModelCheckpoint(filepath= MODEL_PATH, monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True)

# early_stopping
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True)

# 모델 학습
history = model.fit(train_generator,
                    steps_per_epoch=20,
                    epochs=200,
                    validation_data=validation_generator,
                    validation_steps=10,
                    callbacks=[cp,es], 
                    batch_size=10,)

@ 모델저장하기
from keras.models import load_model

model.save('/content/drive/MyDrive/AIVLE3rd_individual/model/[개인]미니프로젝트3차_A021087.h5')

* 모델 SAVE

In [ ]:
import h5py
name = '김나현'
model.save(MODEL_PATH + '/미니프로젝트3차_' + name + '.h5')
print(MODEL_PATH + '/미니프로젝트3차_' + name + '.h5')

6) 모델적용 예측결과

In [ ]:
def my_model_predict(test_generator, model):
  import datetime as df
  import numpy as np
  import pandas as pd

  class_map = {
    0: 'jung',
    1: 'park',    
    2: 'sam',
    3: 'sayuri',
  }

  keys = ['predict_v1','predict_v2','predict_v3']
  predict_verList = dict.fromkeys(keys)

  predict = model.predict(test_generator)

  np.set_printoptions(formatter={'float':lambda x:"{0:0.3f}".format(x)})
  predict_v1 = pd.DataFrame({
      'file' : test_generator.filenames,
      class_map[0] : predict[:,0],
      class_map[1] : predict[:,1],
      class_map[2] : predict[:,2],
      class_map[3] : predict[:,3],
      })
  
  pd.options.display.float_format = '{:.2f}'.format
  predict_verList['predict_v1'] = predict_v1


  return predict_verList

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('/content/drive/MyDrive/AIVLE3rd_individual/model/[개인]미니프로젝트3차_A021087.h5', 
                                custom_objects=None, compile=True)

In [ ]:
predict_verList = my_model_predict(test_generator, model)

In [ ]:
print('모델결과_v1')
predict_verList['predict_v1']

7) 모델 평가

In [ ]:
from sklearn.metrics import f1_score

pred = model.predict(validation_generator)

pred = np.argmax(pred, axis = 1 )

f1 = f1_score(validation_generator.labels, pred, average = None)
print('f1 score :', f1)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

if not isinstance(history, dict):
    history = history.history

plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('Accuracy : Training vs Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc=0)
plt.show()

if not isinstance(history, dict):
    history = history.history

plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Loss : Training vs Validation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc=0)
plt.show()

### UltraLytics YOLO v3 Image Detection

In [ ]:
1) UltraLytics git에서 복사하기
!git clone https://github.com/ultralytics/yolov3.git

2) yolov3 폴더 이동 및 requirements.txt 내부 패키지 설치
!cd yolov3; pip install -r /content/yolov3/requirements.txt

3) Image Detection
3-1) 예제 이미지 다운로드 => !wget -O [저장할 파일명] [파일 주소]
!wget -O /content/yolov3/data/images/14th_street.jpg\
https://raw.githubusercontent.com/DrKAI/image/main/14th_Street_2005.jpg

3-2) COCO Dataset(성능지표)으로 pretrained 된 weights 다운로드 [weights가 없으면 자동 다운로드] 
=> !mkdir [경로/디렉토리 명]
=> pretrained weights 다운로드
!mkdir /content/yolov3/pretrained
!wget -O /content/yolov3/pretrained/yolov3-tiny.pt\
https://github.com/ultralytics/yolov3/releases/download/v9.6.0/yolov3-tiny.pt

3-3) detect.py를 python으로 직접 호출하여 수행 => 명령어 도움말 : !cd yolo3; python detect.py -h
!cd yolov3; python detect.py -h
!cd yolov3; python detect.py \
    --weights '/content/yolov3/pretrained/yolov3.pt' \
    --source '/content/yolov3/data/images' \
    --project '/content/yolov3/detected' \
    --name 'images' \
    --img 640 \
    --conf-thres 0.1 \
    --iou-thres 0.4 \
    --line-thickness 2 \
    --exist-ok \# 덮어쓰기
    --device CPU

@ Detect Image 살펴보기
from IPython.display import Image
from google.colab import files

# Image(filename=[파일 경로])
Image(filename='/content/yolov3/detected/images/14th_street.jpg', width=850)

# files.download(filename=[파일 경로])
files.download(filename='/content/yolov3/detected/images/14th_street.jpg')

# zip
!zip -r /content/detected_image.zip /content/yolov3/detected/images2

### UltraLytics_YOLOv3_VideoDetection

In [ ]:
!git clone https://github.com/ultralytics/yolov3.git

!cd yolov3; pip install -r /content/yolov3/requirements.txt

!mkdir /content/yolov3/data/videos
!wget -O /content/yolov3/data/videos/noway.mp4\
https://github.com/DrKAI/image/raw/main/No_Way_This_Happened.mp4

!mkdir /content/yolov3/pretrained
!wget -O /content/yolov3/pretrained/yolov3.pt\
https://github.com/ultralytics/yolov3/releases/download/v9.6.0/yolov3.pt

# !cd yolov3; python detect.py -h
!cd yolov3; python detect.py \
    --weights '/content/yolov3/pretrained/yolov3.pt' \
    --source '/content/yolov3/data/videos/' \
    --project '/content/yolov3/detected' \
    --name 'videos' \
    --img 640 \
    --conf-thres 0.5 \
    --iou-thres 0.4 \
    --line-thickness 2 \
    --exist-ok
    # --device CPU
    
from google.colab import files
## colab은 멀티 다운로드 지원X
## 폴더 압축하여 파일 하나로 만들고 다운로드
!zip -r /content/detected_videos.zip /content/yolov3/detected/videos/

files.download(filename='/content/yolov3/detected/videos/')

### UltraLytics_YOLOv5_CustomData_ImageDetection

In [ ]:
!git clone https://github.com/ultralytics/yolov5
    
!cd yolov5; pip install -r requirements.txt

@ Image Detection
1) 사전 작업된 CustomData yaml 다운로드
!wget -O /content/yolov5/data/street.yaml\
https://raw.githubusercontent.com/DrKAI/CV/main/street_example.yaml

2) pretrained 된 weights 다운로드 => weights가 없으면 자동 다운로드
!mkdir /content/yolov5/pretrained
!wget -O /content/yolov5/pretrained/yolov5s.pt\
https://github.com/ultralytics/yolov5/releases/download/v6.2/yolov5s.pt

3) train용 이미지 다운로드
!mkdir /content/datasets; mkdir /content/datasets/street
!mkdir /content/datasets/street/images; mkdir /content/datasets/street/images/train
!mkdir /content/datasets/street/labels; mkdir /content/datasets/street/labels/train

!wget -O /content/street_images1.zip https://github.com/DrKAI/CV/raw/main/street_images1.zip
!wget -O /content/street_images2.zip https://github.com/DrKAI/CV/raw/main/street_images2.zip
!wget -O /content/street_images3.zip https://github.com/DrKAI/CV/raw/main/street_images3.zip

!wget -O /content/street_labels1.zip https://github.com/DrKAI/CV/raw/main/street_labels1.zip
!wget -O /content/street_labels2.zip https://github.com/DrKAI/CV/raw/main/street_labels2.zip
!wget -O /content/street_labels3.zip https://github.com/DrKAI/CV/raw/main/street_labels3.zip
    
!unzip /content/street_images1.zip -d /content/datasets/street/images/train
!unzip /content/street_images2.zip -d /content/datasets/street/images/train
!unzip /content/street_images3.zip -d /content/datasets/street/images/train

!unzip /content/street_labels1.zip -d /content/datasets/street/labels/train
!unzip /content/street_labels2.zip -d /content/datasets/street/labels/train
!unzip /content/street_labels3.zip -d /content/datasets/street/labels/train

4) train.py 실행
!cd yolov5; python train.py -h
!cd yolov5; python train.py \
    --data '/content/yolov5/data/street.yaml' \
    --cfg '/content/yolov5/models/yolov5s.yaml' \
    --weights '/content/yolov5/pretrained/yolov5s.pt' \
    --epochs 1000 \
    --patience 7 \
    --img 640 \
    --project 'trained' \
    --name 'train_street' \
    --exist-ok
    # --device cpu

5) detect용 이미지 다운로드
!wget -O /content/yolov5/data/images/street01.jpeg https://github.com/DrKAI/image/raw/main/001.jpeg
!wget -O /content/yolov5/data/images/street02.jpg https://github.com/DrKAI/image/raw/main/14th_Street_2005.jpg
!wget -O /content/yolov5/data/images/street03.jpg https://github.com/DrKAI/image/raw/main/street02.jpg
!wget -O /content/yolov5/data/images/street04.jpg https://github.com/DrKAI/image/raw/main/street03.jpg
!wget -O /content/yolov5/data/images/street05.jpg https://github.com/DrKAI/image/raw/main/street04.jpg
!wget -O /content/yolov5/data/images/street06.jpg https://github.com/DrKAI/image/raw/main/street05.jpg


6) detect.py 실행
!cd yolov5; python detect.py -h

!cd yolov5; python detect.py \
    --weights '/content/yolov5/trained/train_street/weights/best.pt' \
    --source '/content/yolov5/data/images/' \
    --project '/content/yolov5/detected' \
    --name 'images' \
    --img 640 \
    --conf-thres 0.25 \
    --iou-thres 0.5 \
    --line-thickness 2 \
    --exist-ok 
    # --device CPU

7) Detect Image 살펴보기
from IPython.display import Image
from google.colab import files

Image(filename='/content/yolov5/detected/images/street01.jpeg', width=640)

## colab은 멀티 다운로드를 지원하지 않는다
## 폴더를 압축하여 파일 하나로 만들고 다운로드 한다

!zip -r /content/detected_images.zip /content/yolov5/detected/images

files.download(filename='/content/detected_images.zip')

## 크롤링

In [ ]:
requests 이용
받아오는 문자열에 따라 두가지 방법으로 구분
json 문자열로 받아서 파싱하는 방법 : 주로 동적 페이지 크롤링할때 사용
html 문자열로 받아서 파싱하는 방법 : 주로 정적 페이지 크롤링할때 사용
selenium 이용
브라우져를 직접 열어서 데이터를 받는 방법
크롤링 방법에 따른 속도
requests json > requests html > selenium
# summary
# web : server-client : url
# request, response : get,post
# 웹서비스의 구조
# 웹페이지의 종류
# - 동적페이지 : URL 변경 X > 데이터 수정 : JSON : API
# - 정적페이지 : URL 변경 O > 데이터 수정 : HTML : css selector > BeautifulSoup : select(), select_one()


# html : 웹페이지에서 레이아웃, 텍스트 등의 데이터를 작성
# 구성요소: document, element, tag, attr, text
# element 계층적 구조
# tag 종류 : p, span, ul, li, table, a, img, iframe, div

# css selector : html의 element에 style을 적용시킬때 element를 선택하는 방법
# element 선택 : tag(span), class(.), id(#), attr([value="no1"])
# n번째 element 선택 : .py:nth-child(2) : 2번째 엘리먼트중에 클래스가 py인 엘리먼트
# 계층적 element 선택 : 모든 하위 엘리먼트 선택(.wrap p), 한단계 하위 엘리먼트 선택(.wrap > p), 여러개 선택(.no1, .no2)

### Selenium
- 브라우져의 자동화 목적으로 만들어진 다양한 브라우져와 언어를 지원하는 라이브러리
- 브라우져를 파이썬 코드로 컨트롤 해서 브라우져에 있는 데이터를 수집

#### 크롤링 방법
- 1. requests : json : 웹페이지의 API 트래픽을 분석해서 데이터 수집 : naver stock
- 2. requests : json : 공식적으로 제공하는 API를 application key 받아서 데이터 수집 : naver api(papago, trend)
- 3. requests : html, BeautifulSoup(css selector) : 웹페이지의 html 코드 받아서 데이터 수집 : gmarket
- 4. selenium : browser - python : 브라우져를 파이썬 코드로 컨트롤 해서 데이터 수집 : ted
- 크롤링할때 좋은 순서 : 2 > 1 > 3 > 4

### 동적페이지

In [ ]:
# 동적 페이지 데이터 수집 프로세스
# 1. 웹서비스 분석(개발자도구) : URL
# 2. request(url, params, header) > response(json) : JSON(str)
# 3. JSON(str) > list, dict > DataFrame

In [ ]:
import requests
import pandas as pd

1) pc 웹페이지가 복잡하면 mobile 웹페이지에서 수집
2) 서버에 데이터 요청 : request(url) > response : json(str)
- response의 status code가 200이 나오는지 확인
- 403이나 500이 나오면 request가 잘못되거나 web server에서 수집이 안되도록 설정이 된것임
  -> header 설정 또는 selenium 사용
- 200이 나오더라도 response 안에 있는 내용을 확인 > 확인하는 방법 : response.text[:200]
3) 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame
4) 함수로 만들기

def stock_price(pagesize, page, code="KOSPI"):
    """This function is crawling stock price form naver webpage.
    Params
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI or KOSDAQ
    
    Return
    ------
    type : DataFrame : display date, price columns
    """
    url = f"https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}"
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[["localTradedAt","closePrice"]]

kospi = stock_price(60,1,"KOSPI")
kosdaq = stock_price(60,1,"KOSDAQ")

# 데이터 전처리
df = kospi.copy()
df["kosdaq"]=kosdaq["closePrice"]
df["usd"] = usd['closePrice']
df = df.rename(columns={"closePrice":"kospi"})
df

# docstring[""""""] : 함수를 사용하는 방법을 문자열로 작성 
#help()이용, Shift + tab
help(stock_price)

df = pd.DataFrame([{'age': 23},{'age': 36},{'age': 27}])  # 데이터프레임 만들기
df

@ 시각화
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 수집
page_size = 60
kospi_df = stock_price("KOSPI", page_size=page_size)
kosdaq_df = stock_price("KOSDAQ", page_size=page_size)
usd_df = exchage_rate("FX_USDKRW", page_size=page_size)
eur_df = exchage_rate("FX_EURKRW", page_size=page_size)

# 데이터 전처리 1 : 데이터 타입 변경
print(kospi_df.dtypes)
kospi_df["kospi"] = kospi_df["closePrice"].apply(lambda data: float(data.replace(",", "")))
kospi_df = kospi_df.drop(columns=["closePrice"])
print(kospi_df.dtypes)

kosdaq_df["kosdaq"] = kosdaq_df["closePrice"].apply(lambda data: float(data.replace(",", "")))
usd_df["usd"] = usd_df["closePrice"].apply(lambda data: float(data.replace(",", "")))
eur_df["eur"] = eur_df["closePrice"].apply(lambda data: float(data.replace(",", "")))

kosdaq_df = kosdaq_df.drop(columns=["closePrice"])
usd_df = usd_df.drop(columns=["closePrice"])
eur_df = eur_df.drop(columns=["closePrice"])

# 데이터 전처리 2 : 날짜 데이터 맞추기 : merge
merge_df_1 = pd.merge(kospi_df, kosdaq_df, on="localTradedAt")
merge_df_2 = pd.merge(merge_df_1, usd_df, on="localTradedAt")
merge_df_3 = pd.merge(merge_df_2, eur_df, on="localTradedAt")
merge_df = merge_df_3.copy()
merge_df.tail(2)

# 시각화
plt.figure(figsize=(20, 5))
columns = merge_df.columns[1:]
for column in columns:
    plt.plot(merge_df["localTradedAt"], merge_df[column], label=column)

xticks_count = 11
plt.xticks(merge_df["localTradedAt"][::int(len(merge_df) // xticks_count) + 1])
plt.legend(loc=0)
plt.show()

corr_df = merge_df[merge_df.columns[1:]].corr()
corr_df

# 결정계수 : r-squared 
# 1과 가까울수록 강한 관계, 0과 가까울수록 약한 관계
plt.figure(figsize=(20, 5))
sns.heatmap(corr_df**2, cmap="YlGnBu", annot=True)
plt.show()

In [ ]:
# 컬럼의 데이터 타입 변경 : str > float
# df[column].apply() : 모든 데이터를 함수에 대입한 결과를 출력
# apply(func) : 모든 데이터를 func을 적용시킨 결과 출력
df.dtypes

# 피어슨 상관계수 : df.corr()
# 1과 가까울수록 강한 양의 상관관계를 갖는다.
# -1과 가까울수록 강한 음의 상관관계를 갖는다.
# 0과 가까울수록 관계가 없다.
df[['kospi','kosdaq','usd']].corr()

### 카카오 API

In [ ]:
import requests
import pandas as pd



REST_API_KEY = '9a2d4ed549676750779492f1f9f89956'



url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
#params = {query': '}
params = {'x' : '127.177375','y' : '37.240666'}
headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}



response = requests.get(url, params = params, headers=headers)
response

### API

In [ ]:
application programing interface
api 를 사용해서 데이터를 수집하는것은 서비스에 데이터를 제공한는 공식적인 방법으로 데이터 수집

In [ ]:
# API를 이용한 데이터 수집
# 1. APP 등록 : application key 
# 2. API 문서 : URL
# 3. request(url, params, header(application key)) > response(json) : JSON(str)
# 4. JSON(str) > list, dict > DataFrame or Text

In [ ]:
import pandas as pd
import requests, json

In [ ]:
CLIENT_ID, CLIENT_SECRET = "KE2M4YcO4Sv9P7HrFXJ1", "9X0zM51ZRf"

# json.dumps() : 인터넷 트래픽에서는 영문, 숫자, 특수문자만 사용가능
# 한글과 같은 문자를 인코딩(영문,숫자,특수문자)
txt= "파이썬은 재미있습니다."
url = "https://openapi.naver.com/v1/papago/n2mt" 
params = {"source": "ko", "target": "en", "text": txt}
headers = {"Content-Type": "application/json", 
           "X-Naver-Client-Id": CLIENT_ID,
           "X-Naver-Client-Secret": CLIENT_SECRET}

response = requests.post(url, json.dumps(params),headers=headers)
response

response.text

txt_en = response.json()["message"]["result"]["translatedText"]
txt_en

# 이렇게 가져올수 있다[다음 환율]
datas = response.json()["data"]
df = pd.DataFrame(datas)
df.head(1)

def translate(txt):
    CLIENT_ID, CLIENT_SECRET = "KE2M4YcO4Sv9P7HrFXJ1","9X0zM51ZRf" 
    url = "https://openapi.naver.com/v1/papago/n2mt"
    params = {"source": "ko", "target": "en", "text": txt}
    headers = {"Content-Type": "application/json", 
           "X-Naver-Client-Id": CLIENT_ID,
           "X-Naver-Client-Secret": CLIENT_SECRET}
    response = requests.post(url, json.dumps(params), headers=headers)
    txt_en = response.json()["message"]["result"]["translatedText"]
    return txt_en

txt= "웹크롤링은 재미있습니다."
txt_en = translate(txt)
txt_en

@ 한글 excel 파일을 영문 excel 파일로 변경

%ls

covid = pd.read_excel("covid.xlsx")[["category","title"]]
covid.tail(2)

covid_en = covid["title"].apply(translate)

covid["title_en"] = covid_en
covid

# utf-8-sig : excel에서 사용하는 인코딩 방식과 호환이 되는 utf-8인 인코딩 방식 
covid.to_excel("covid_en.xlsx", index = False, encoding="utf-8-sig")

### 통합검색어 트렌드 api

In [ ]:
# 1. URL
url = "https://openapi.naver.com/v1/datalab/search"

# 2. request > response
params = {
    "startDate": "2018-01-01",
    "endDate": "2022-01-31",
    "timeUnit": "month",
    "keywordGroups": [
        {"groupName": "트위터", "keywords": ["트위터", "트윗"]},
        {"groupName": "페이스북", "keywords": ["페이스북", "페북"]},
        {"groupName": "인스타그램", "keywords": ["인스타그램", "인스타"]},
    ]
}

headers = {
    "Content-Type": "application/json",
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret": CLIENT_SECRET,    
}

response = requests.post(url, data=json.dumps(params), headers=headers)
response

# 3. parsing
datas = response.json()["results"]

# 4. preprocessing
result_df = pd.concat(dfs, ignore_index=True)
result_df.tail(2)

pivot_df = result_df.pivot("period", "title", "ratio")
pivot_df.columns = ["instagram", "twitter", "facebook"]
pivot_df.tail(2)

# 5. visualization

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import matplotlib.pyplot as plt

pivot_df.plot(figsize=(20, 5))
plt.legend(loc=0)
plt.show

### 위도 경도로 찾기

- 절차
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

In [ ]:
# 1. 동이름으로 위도 경도 구하기
addr = "망원동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

# 2. 위도 경도로 geohash 알아내기
# install geohash2
# !pip install geohash2
import geohash2

# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

# 3. geohash로 매물 아이디 가져오기
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
response = requests.get(url)
datas = response.json()["items"]
# len(datas), datas[0]
ids = [data["item_id"] for data in datas]
len(ids), ids[:5]

# 4. 매물 아이디로 매물 정보 가져오기
# 1000개 넘어가면 나눠서 수집해야 함
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": ids
}
response = requests.post(url, params)
response

datas = response.json()["items"]
df = pd.DataFrame(datas)
df.tail(2)

# 필요한 컬럼만 필터링
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "floor", "building_floor",
           "address1", "manage_cost"]
filtered_column_df = df[columns]
filtered_column_df.tail(2)

# 주소에 망원동이 있는 데이터만 필터링
result_df = filtered_column_df[filtered_column_df["address1"].str.contains("망원동")]
result_df = result_df.reset_index(drop=True)
result_df.tail(2)

### 이미지 크롤링

In [ ]:
import pandas as pd
import requests, os

# 1. 디렉토리 생성 : data
path = "data"
if not os.path.exists("data"): # 디렉토리 존재 유무 확인
    os.makedirs(path)
    
%ls
# 2. csv 파일을 로드 : image link
df = pd.read_csv("gmarket.csv")
df.tail(2)

img_link = df.loc[0, 'img']
img_link

# 3. download images : requests
response = requests.get(img_link)
response

with open(f'{path}/test.png', "wb") as file: # 파일저장 wb, 읽기는 rd
    file.write(response.content)
    
%ls data

# 4. display image : pillow
from PIL import Image as pil
pil.open(f'{path}/test.png') 

# 5. 여러개의 이미지 다운로드
df[:3]
for idx, data in df[:5].iterrows():
    filename = '0' * (3 - len(str(idx))) + str(idx) + '.png'
    print(idx, filename, data['img'])
    response = requests.get(data['img'])
    with open(f'{path}/{filename}', "wb") as file: # 파일저장 wb, 읽기는 rd
        file.write(response.content)

pil.open(f'{path}/004.png')


## 언어지능 딥러닝

### MeCab

In [ ]:
#<MeCab = 형태소 분석기>
from google.colab import drive
drive.mount('/content/gdrive')

# MeCab 설치하기
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
    
!pwd
%cd Mecab-ko-for-Google-Colab

!bash install_mecab-ko_on_colab_light_220429.sh 

# 실행 테스트
import MeCab

tagger = MeCab.Tagger()

sentence = '코로나 감염 뒤 4주 이상 후유증이 이어지는 현상을 롱코비드라고 부른다.'
print (tagger.parse(sentence))

# 설치 후 Tagger 에러가 발생한 경우
# https://github.com/konlpy/konlpy/issues/144
#!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
#import os
#os.chdir('mecab-python-0.996')
#!python setup.py build
#!python setup.py install

!pwd

@ MeCab 사용자 사전 추가하기

# Mecab의 사전 디렉토리
%cd /content/mecab-ko-dic-2.1.1-20180720/
ls

!ls user-dic/

# 사용자 사전에 미리 등록되어있는 고유명사
!cat user-dic/nnp.csv

1.Colab 안에서 직접 단어 추가하기
# 사전에 등록할 단어 추가
!echo "코로나,,,,NNP,*,T,코로나,*,*,*,*,*" >> user-dic/nnp.csv
!cat user-dic/nnp.csv

mv user-dic/nnp.csv /content/gdrive/'My Drive'/'aivle'

2.텍스트 파일로 로컬에서 수정 후 추가하기
# nnp.csv 파일 이동
!mv /content/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv /content/gdrive/'My Drive'/'aivle'

로컬에서 nnp.csv 파일에 단어를 추가한 후에 구글드라이브에 업로드
# 로컬에서 파일 수정 후 재 업로드
#!cat /content/gdrive/'My Drive'/'Colab Notebooks'/nng.csv
#!cat /content/gdrive/'My Drive'/'Colab Notebooks'/nnp.csv
!cat /content/gdrive/'My Drive'/'Colab Notebooks'/aivle/DAY1/nng.csv
!cat /content/gdrive/'My Drive'/'Colab Notebooks'/aivle/DAY1/nnp.csv

sentence = '토트넘이 리그 4위로 21-22 프리미어리그 시즌을 마무리해서 UCL에 진출했다.'
print (tagger.parse(sentence))

# 업로드된 수정 파일 이동
!mv /content/gdrive/'My Drive'/'Colab Notebooks'/aivle/DAY1/nng.csv /content/mecab-ko-dic-2.1.1-20180720/user-dic/
!mv /content/gdrive/'My Drive'/'Colab Notebooks'/aivle/DAY1/nnp.csv /content/mecab-ko-dic-2.1.1-20180720/user-dic/
!ls user-dic/

!cat /content/mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv
!cat /content/mecab-ko-dic-2.1.1-20180720/user-dic/nng.csv

# 사용자사전 업데이트
!bash ./tools/add-userdic.sh

# 사전 리빌드
!sudo make install

tagger = MeCab.Tagger()
# 사용자 사전 업데이트 확인
sentence = '코로나 감염 뒤 4주 이상 후유증이 이어지는 현상을 롱코비드라고 부른다.'
print (tagger.parse(sentence))

### word_count

In [ ]:
# colab 환경에 Nanum 폰트를 설치
!sudo apt-get install -y fonts-nanum
# scans the font directories and build font cache
!sudo fc-cache -fv
# matplotlib의 font cache를 clear
!rm ~/.cache/matplotlib -rf
# 실행 후 런타임을 다시 시작하세요

from google.colab import drive
drive.mount('/content/gdrive')

!ls /content/gdrive/'My Drive'/aivle/data/

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
    
#!cd Mecab-ko-for-Google-Colab
!pwd
%cd Mecab-ko-for-Google-Colab

!bash install_mecab-ko_on_colab_light_220429.sh

import MeCab
import pandas as pd
import numpy as np

tagger = MeCab.Tagger()
with open('/content/gdrive/My Drive/aivle/data/extreme_job_review.txt','r',encoding='utf-8') as f:
    my_sentence=f.read()
my_sentence

print(tagger.parse(my_sentence))

# Mecab 형태소분석 결과에서 단어부분(스트링)와 품사태그 부분을 분리해서 반환해주는 함수
# 예) 영화배우/NNG -> ('영화배우', 'NNG')
def mecabsplit(mecab_tagger,inputs, pos):
    r=[]
    inputs = mecab_tagger.parse(inputs)
    t = inputs.split('\n')[:-2]
    for i in t:
        field = i.split('\t')
        if field[1].split(',')[-1] is not '*':
            r.extend( [ (x.split('/')[0],x.split('/')[1]) for x in field[1].split(',')[-1].split('+') ] )
        else:
            r.append( (field[0],field[1].split(',')[0]) )
    if pos:
        return r
    else:
        return [ x[0] for x in r ]
    
# tag가 NNG(일반 명사)인 단어 (토큰, 형태소) 리스트
nng = [word for word, tag in mecabsplit(tagger, my_sentence, True) if tag=='NNG']
print(nng)

# tag가 NNP(고유 명사)인 단어 (토큰, 형태소) 리스트
nnp = [word for word, tag in mecabsplit(tagger, my_sentence, True) if tag=='NNP']
print(nnp)

from collections import Counter,OrderedDict

# matplotlib의 폰트를 Nanum 폰트로 지정
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

count_list=Counter(nng)
print(len(count_list))

# counter안의 value를 sort해줌
sorted_list=count_list.most_common(20)
# 다시 dictionary 형태로 변환
sorted_list=OrderedDict(sorted_list)

plt.figure(figsize=(20, 10))
plt.xticks(rotation=90)
plt.plot(list(sorted_list.keys()), list(sorted_list.values()))
plt.title("일반 명사 빈도수")
plt.ylabel("# of words")
plt.xlabel("word")
plt.show()

### sentiment_BoW[Naive Bayes]

In [ ]:
# MeCab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
    
%cd Mecab-ko-for-Google-Colab/

!bash install_mecab-ko_on_colab_light_220429.sh 

#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

# Mecab 형태소분석 결과에서 단어부분(스트링)와 품사태그 부분을 분리해서 반환해주는 함수
# 예) 영화배우/NNG -> ('영화배우', 'NNG')
def mecabsplit(mecab_tagger,inputs, pos):
    r=[]
    inputs = mecab_tagger.parse(inputs)
    t = inputs.split('\n')[:-2]
    for i in t:
        field = i.split('\t')
        if field[1].split(',')[-1] is not '*':
            r.extend( [ (x.split('/')[0],x.split('/')[1]) for x in field[1].split(',')[-1].split('+') ] )
        else:
            r.append( (field[0],field[1].split(',')[0]) )
    if pos:
        return r
    else:
        return [ x[0] for x in r ]
    
# utf-8 인코딩으로 된 한글 파일을 읽어들임
import codecs
def read_data(filename):
    with codecs.open(filename, 'r', 'utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

# 데이터 파일 위치 확인하세요
train_data = read_data('/content/gdrive/My Drive/aivle/data/nsm/small_ratings_train.txt')
test_data = read_data('/content/gdrive/My Drive/aivle/data/nsm/small_ratings_test.txt')

print(len(train_data))
print(len(test_data))
print (train_data[0])
print (test_data[0])


%%time
# 형태소 분석기 Mecab import
import MeCab
tagger = MeCab.Tagger()

# Mecab의 출력 스트링을 token+space+token+space ... 포맷으로 변환해줌 (pos=False일 때)
def tokenize(doc):
    return ' '.join(mecabsplit(tagger, doc, False))

# train data의 첫번째 줄
print (train_data[0])
# test data의 첫번째 줄
print (test_data[0])

# train data의 document 부분
train_docs_X = [tokenize(row[1]) for row in train_data]
# train data의 label 부분
train_Y = [row[2] for row in train_data]

test_docs_X = [tokenize(row[1]) for row in test_data]
test_Y = [row[2] for row in test_data]

# train data 확인
print(train_docs_X[0])
print(train_Y[0])
print(train_docs_X[1])
print(train_Y[1])
print('\n')
# test data 확인
print(test_docs_X[0])
print(test_Y[0])
print(test_docs_X[1])
print(test_Y[1])
print('\n')

@ CountVectorizer 변환
데이터 : ['I am happy', 'I am sad']
단어 모음 : ['I', 'am', 'happy', 'sad']
변환 결과 : [[1, 1, 1, 0], [1, 1, 0, 1]]
    
# CountVectorizer 기능: 
# 1. 문서를 토큰 리스트로 변환한다
# 2. 각 문서에서 토큰의 빈도를 센다
# 3. 각 문서를 BoW 벡터로 변환한다.
from sklearn.feature_extraction.text import CountVectorizer


# max_features : 전체 문서에서 빈도수 높은 순으로 top max_features 갯수의 단어만 포함하여
# fit : vocabulary를 생성한다
vec = CountVectorizer(max_features = 1000).fit(train_docs_X)

# 학습데이터(train_docs_X), 테스트데이터(test_docs_X)를 BoW 벡터로 변환
train_X = vec.transform(train_docs_X).toarray()
test_X = vec.transform(test_docs_X).toarray()

# 첫번째 문서(리뷰 문장)의 벡터
print(train_X[0])
# 학습데이터 문서(리뷰 문장) 갯수
print(len(train_X))


@ Naive Bayes 모델 이용하기
%%time

# sklearn 팩키지에서 GaussianNB 모델을 import
from sklearn.naive_bayes import GaussianNB

# GaussianNB 객체를 생성
gnb = GaussianNB()

# train_X: 학습데이터, train_Y: 정답레이블을 받아서 학습
gnb.fit(train_X, train_Y)

# score(test_data, true_labels) returns the mean accuracy on the given test data and labels.
print("Accuracy on training set: {:.3f}".format(gnb.score(train_X, train_Y)))
print("Accuracy on test set: {:.3f}".format(gnb.score(test_X, test_Y)))


pos = [ [test_docs_X[i], test_Y[i], gnb.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '1' ]
neg = [ [test_docs_X[i], test_Y[i], gnb.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '0' ]
#crr= []
#for i in range(len(test_Y)):
#    if test_Y[i] == '1':
#        crr.append([test_docs_X[i], test_Y[i], gnb_predict([test_X[i]]) ])
#print (crr)
print ('>긍정 리뷰에 대한 예측:')
for i in pos[:5]:
    print ('입력:', i[0])
    print ('정답:', i[1])
    print ('예측:', i[2])
print ('\n>부정 리뷰에 대한 예측:')
for i in neg[:5]:
    print ('입력:', i[0])
    print ('정답:', i[1])
    print ('예측:', i[2])

    
@  KNN 모델 이용하기
# (연습 1) KNN으로 긍부정 분류를 하세요. (sklearn 라이브러리에서 제공하는 KNeighborsClassifier를 사용)
# write code here
# 
#
#
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(train_X, train_Y)

# (연습 2) score 함수를 사용하여 학습데이터의 정확도, 테스트데이터의 정확도를 출력하세요.
# 소수점 이하 3자리까지 출력하세요.
#
#
print("Accuracy on training set: {:.3f}".format(neigh.score(train_X, train_Y)))
print("Accuracy on test set: {:.3f}".format(neigh.score(test_X, test_Y)))

# KNN으로 예측한 결과를 확인하기
pos = [ [test_docs_X[i], test_Y[i], neigh.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '1' ]
neg = [ [test_docs_X[i], test_Y[i], neigh.predict([test_X[i]]) ] for i in range(len(test_Y)) if test_Y[i] == '0' ]

print ('>긍정 리뷰에 대한 예측:')
for i in pos[:5]:
    print ('\n입력:', i[0])
    print ('정답:', i[1])
    print ('예측:', i[2])
print ('\n>부정 리뷰에 대한 예측:')
for i in neg[:5]:
    print ('\n입력:', i[0])
    print ('정답:', i[1])
    print ('예측:', i[2])

### sentiment_vader

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# 사전 기반 감성분석 툴 vaderSentiment 설치
!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
    
sentiment_analyzer_scores("The phone is super cool.")

# 아마존 상품리뷰
import pandas as pd

df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/aivle/data/data_sentiment.csv',encoding='utf-8')
print(df.head())

# vader 결과
vaderresult=[]
# 상품리뷰 점수와 같은 경우
truecount = 0

for i in range(df.id.count()):
    # compound결과가 0.05보다 크면 긍정, -0.05보다 작으면 부정
    if analyser.polarity_scores(df.textcontent[i]).get("compound")>0.05:
        vaderresult.append(1)
    elif analyser.polarity_scores(df.textcontent[i]).get("compound")< -0.05:
        vaderresult.append(0)
    else:
        vaderresult.append(3)

    print(df.textcontent[i], df.reviewrating[i], vaderresult[i])

    #자세히 보고 싶다면..
    #print(df.textcontent[i],analyser.polarity_scores(df.textcontent[i]))
    
for i in range(df.id.count()):
    if vaderresult[i]==df.reviewrating[i]:
        truecount=truecount+1
# 정확도
print("정확도 : ", truecount/(df.id.count()))
print("정확도 : ", format(truecount/(df.id.count()), ".3f"))
#print("정확도 : ", round(truecount/(df.id.count()), 3))

### clustering_dbscan

In [ ]:
#구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# 입력 데이터 위치
Path = '/content/drive/MyDrive/aivle/data/clustering/' 

# 각 주제별 100건의 뉴스 데이터
# Read an Excel file into a pandas DataFrame
# DataFrame : 2차원 데이터구조. Row, Column, Series(각 Column에 있는 데이터들)로 구성
climate = pd.read_excel(Path+'news_climate change.xlsx')
mobility = pd.read_excel(Path+'news_mobility.xlsx')
probiotics = pd.read_excel(Path+'news_probiotics.xlsx')

# 주제 column을 추가
climate['주제'] = '기후위기'
mobility['주제'] = '모빌리티'
probiotics['주제'] = '유산균'

# 각 주제별 70건의 뉴스데이터를 가져와서 입력용 데이터를 생성
# pandas.concat(objs, axis=0, ignore_index=Faslse, ...)
# axis=0, 0: 위+아래로 합치기, 1: 왼쪽+오른쪽으로 합치기
# ignore_index=True : 기존 index를 무시. The resulting axis will be labeled 0, …, n - 1. 
data = pd.concat([climate[:70], mobility[:70], probiotics[:70]], ignore_index=True)

# 뉴스기사 body, title을 기준으로 중복을 제거
# DataFrame.drop_duplicates : Returns DataFrame with duplicate rows removed.
data_unique = data.drop_duplicates(['body']).drop_duplicates(['title'])

# 중복 제거된 데이터 건수를 확인
# DataFrame.shape : Returns a tuple representing the dimensionality of the DataFrame.
data.shape, data_unique.shape

@ 중복으로 들어있는 기사를 제거하고 기사들의 순서를 무작위로 섞습니다.

# 데이터 순서를 셔플링
# frac: 랜덤 추출할 비율 (1 == 전체 데이터를 셔플링)
# random_state: 랜덤 추출할 값에 seed 설정하면, 항상 같은 결과를 생성
# reset_index: Reset the index. Use drop parameter to avoid the old index being added as a column
data_shuffled = data_unique.sample(frac=1, random_state=16).reset_index(drop=True) 
data_shuffled.tail(5)

# DataFrame.iloc : integer-location based indexing for selection by position
data_shuffled.iloc[0]['title']
data_shuffled.iloc[0]['body']

# 한국어 형태소 분석기를 포함하고 있는 파이썬 패키지 설치
!pip install konlpy

< 뉴스 기사 단어 빈도 분석 >

# 데이터에서 첫번째 기사 내용을 가져와서 명사만 추출합니다
from konlpy.tag import Hannanum
hannanum = Hannanum()  # 카이스트 연구실에서 개발에서 오픈 

temp = hannanum.nouns(data_shuffled.loc[0]['body'])  # nouns-> 일반명사/ 고유명사만 추출
print(temp)

# 명사만 추출된 리스트
word_list=temp

# 문서에 출현한 각 명사로부터 Series 객체를 생성 (index는 0부터 시작하는 정수값)
# 0 차량
# 1 고장
# 2 제동
# 3 차량
# 4 고장
# 5 ...
word_list=pd.Series([x for x in word_list if len(x)>1])  # 객체생성

# pd.Series.value_counts: Returns a Series containing counts of unique values.
# The resulting object will be in descending order so that the first element is the most frequently-occurring element.  
word_list.value_counts().head(10), data_shuffled.iloc[0]['주제']

< DBSCAN 예제 : 기사 분석하기 >
    
# 기사 내용중 명사만을 추출하여 docs 리스트에 저장합니다.
docs = []
for i in data_shuffled['body']:
    docs.append(hannanum.nouns(i))

# 공백 문자를 넣어서 각 단어를 합칩니다.
for i in range(len(docs)):
    docs[i] = ' '.join(docs[i])

docs[0]

# 처음 한나눔 분석기를 이용해서 명사만 추출한 docs의 구조는 다음과 같은 이중 리스트로 구성되어 있습니다.

# [[문장1], [문장2], ... , [문장70]]
# 두번째 for문을 이용하면 docs는 다음과 같이 변형되고

# [문장1, 문장2, ... , 문장70]
# 각 리스트의 원소가 명사들 사이에 공백이 삽입된 텍스트로 바뀝니다.

# sklearn.feature_extraction.text.TfidfVectorizer :
# Converts a collection of raw documents to a matrix of TF-IDF features (document-term matrix)
# ngram_range(min_n, max_n): all values of n such that min_n <= n <= max_n will be used. 
# min_df : ignore terms that have a document frequency lower than the given threshold.
from sklearn.feature_extraction.text import TfidfVectorizer

# n(1 to 5)gram 을 사용하여 위 기사의 명사들을 tfidf vector로 변환
tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1,5))
tfidf_vectorizer.fit(docs)
vector = tfidf_vectorizer.transform(docs).toarray()
print(vector.shape) # num_of_documents X vocab   DTM (Document -Term Matrix)

# sklearn.cluster.DBSCAN: Performs DBSCAN clustering from vector array
# eps: maximum distance between two samples (default 0.5)
# min_samples: The number of samples in a neighborhood for a point to be considered as a core point (default 5) 
# metric: The metric to use when calculating distance between instances in a feature array. (default 'euclidean')
# 'cosine' distance == (1 - cosine similarity)
# ex) If 2 vectors are perfectly the same then the similarity is 1 (angle=0 hence 𝑐𝑜𝑠(𝜃)=1), and the distance is 0 (1–1=0).
from sklearn.cluster import DBSCAN
import numpy as np

vector = np.array(vector)
model = DBSCAN(eps=0.6, min_samples=3, metric = "cosine") # 얘네를 바꿔서 하면 됨 (노이즈 줄어듦)
result = model.fit_predict(vector) # Computes clusters from a data and predict labels.

# 클러스터 번호 -1에는 노이즈 데이터로 판별되어 클러스터링이 안된 문서들이 들어 있음
print('클러스터 번호: 클러스터에 속한 기사의 수')
result_dict = {str(i):list(result).count(i) for i in set(result)} # result의 각 클러스터에 속한 기사수를 카운트
result_dict # 마지막 159 = 노이즈

# 클러스터 번호를 column으로 추가
data_shuffled['result'] = result

sentences = []
max_cluster_num = 0
for cluster_num in set(result):
    sentence = ''

    # -1은 노이즈 판별이 났거나 클러스터링이 안된 경우
    if(cluster_num == -1): 
        continue
    else:
        max_cluster_num = cluster_num         # 생성된 클러스터의 갯수를 카운트
        temp_df = data_shuffled[data_shuffled['result'] == cluster_num] # cluster num 별로 조회

        print("cluster num : {}".format(cluster_num))
        # zip: 동일한 개수로 이루어진 자료형을 묶어 준다
        for data in zip(temp_df['title'], temp_df['body'], temp_df.iloc): 
            title, body, my_data = data
            print(my_data['주제'], ':', title) # 주제, 기사 제목을 출력
            sentence += body + ' '
        print()
    sentences.append(sentence)
    
# 각 클러스터 마다 데이터 출력해주기
for i in range(max_cluster_num+1):
  data_cluster = data_shuffled[data_shuffled.result==i]
  data_num = len(data_cluster)

  if len(data_cluster) > num_to_display:
    data_cluster = data_cluster[0:num_to_display]

  print('----- 클러스터 %i -----' %i)
  print('총 데이터 개수: %i' % data_num)
  print(data_cluster)
  print()

### Word2Vec_Wiki_Pretrain

In [ ]:
<모델!>
(주의) word2vec 모델 학습에 장시간이 소요되므로 강의 중에 실행하지 않습니다.

from google.colab import drive
drive.mount('/content/gdrive')

from __future__ import print_function

import codecs
import os
import gensim

%%time
def read_data(filename):
    with codecs.open(filename, 'r', 'utf-8') as f:
        data = [line.split(' ') for line in f.read().splitlines()]
    return data
        
data = read_data('/content/gdrive/My Drive/aivle/data/wiki/wiki.txt')

print(len(data))
print(data[0])

%%time
import gensim
# Word2Vec 생성
# sg=1이면 skip-gram 0이면 cbow 방식 사용, min_count 단어 빈도수, size 벡터차원
model = gensim.models.Word2Vec(data, min_count=1, size=100, window=5, sg=1, seed=10)
model.save('/content/gdrive/My Drive/Colab Notebooks/aivle/data/wiki/wiki.w2v')

%%time
model = gensim.models.Word2Vec.load(os.path.join('/content/gdrive/My Drive/aivle/data/wiki', 'wiki.w2v'))

model.wv['고양이']

result = model.wv.accuracy(os.path.join('/content/gdrive/My Drive/aivle/data/wiki','word_analogy_korean.txt')) 
for r in result:
    print('%s\t%s\t%s' % (r['section'], len(r['correct']), len(r['incorrect'])))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# gensim은 텍스트를 벡터로 변환하는 데 필요한 함수들 제공
# gensim is an open-source library for unsupervised topic modeling, document indexing, 
# retrieval by similarity, and other natural language processing functionalities. 
import os
import gensim
import codecs

model = gensim.models.Word2Vec.load(os.path.join('/content/gdrive/My Drive/aivle/data/wiki', 'wiki.w2v'))

model.wv["강아지"]

# Word Analogy Test (임베딩된 단어벡터들이 의미론적/문법적 관계를 잘 반영하는지 테스트)
# Find the top-N most similar words. Positive words contribute positively towards the similarity, negative words negatively.
for t in model.wv.most_similar(positive=["파리", "일본"], negative=["도쿄"], topn=10):
    print('%s\t%f' % (t[0], t[1]))
    
result = model.wv.accuracy(os.path.join('/content/gdrive/My Drive/Colab Notebooks/aivle/data/wiki','word_analogy_korean.txt'))
for r in result:
    print('%s\t%s\t%s' % (r['section'], len(r['correct']), len(r['incorrect'])))
    
# most_similar 함수 top-N개 유사 단어를 반환
for t in model.wv.most_similar("강아지", topn=5):
    print(t[0])
    
model.wv.similarity("호랑이", "표범") # 두 단어의 similar점수 뽑아줌

### PyTorch_tutorial

In [ ]:
# The torch package contains data structures for multi-dimensional tensors and 
# defines mathematical operations over these tensors. 
import torch
import numpy as np

@ 텐서초기화 하기
# The torch package contains data structures for multi-dimensional tensors and 
# defines mathematical operations over these tensors. 
import torch
import numpy as np

# 랜덤값으로 텐서 생성. 정수값으로 텐서의 차원을 전달
# 랜덤값으로 채워진 (3, 4) 차원의 텐서를 생성 
tensor = torch.rand(3, 4)   
print ("tensor :", tensor)
print(f"Shape of tensor: {tensor.shape}")     
print(f"Datatype of tensor: {tensor.dtype}") 
print(f"Device tensor is stored on: {tensor.device}") 

# Returns a tensor filled with the scalar value 1,with the shape defined by the argument size.
tensor = torch.ones(4, 4)
print(tensor)
# 모든 행에 대해서 1번열에 0값을 대입
tensor[:,1] = 0
print(tensor)

# Concatenate : 텐서를 연결하기
# 딥러닝에서는 모델의 입력 또는 중간 연산 단계에서 두 개의 텐서를 연결하는 경우가 있습니다.
# 두 텐서를 연결해서 입력으로 사용하는 것은 두 텐서에 담긴 정보를 모두 사용한다는 의미입니다. 
# dim : 텐서를 연결하여 어느 차원을 늘릴 것인지를 표시
t1 = torch.cat([tensor, tensor], dim=0) # 두 텐서를 연결하여 0번째 차원을 늘리라는 의미
print("tensor shape:", tensor.shape)
print("->", t1)
print("t1 shape:", t1.shape) # 0번째 dimension이 늘어난 것을 확인
print("----------------------------")
t2 = torch.cat([tensor, tensor], dim=1) # 두 텐서를 연결하여 1번째 차원을 늘리라는 의미
print("tensor shape:", tensor.shape)
print("->", t2)
print("t2 shape:", t2.shape) # 1번째 dimension이 늘어난 것을 확인

* 신경망 모델 생성하기

In [ ]:
모든 신경망 클래스는 torch.nn 패키지를 통해서 생성합니다.
torch.nn.Module은 PyTorch의 모든 신경망의 Base Class이며 새로운 신경망 모델은 torch.nn.Module 클래스를 상속하여 정의해야 합니다.
새로운 클래스 내에서 __init()__함수와 forward()함수를 반드시 override 해야 합니다.
__init()__함수에서는 모델에서 사용될 module(nn.Linear, nn.Conv2d), activation function(ReLU 등)를 정의합니다.
forward()에서는 모델에서 실행되어야 하는 연산을 정의합니다.
backward 연산은 backward() 함수를 호출하면 PyTorch가 자동으로 수행하므로 forward()만 정의합니다.
forward()에서는 input 데이터에 대해 어떤 연산을 진행하여 output이 나올지를 정의해 주는 것입니다.

In [ ]:
import torch.nn as nn           # 신경망 구현을 위한 데이터 구조, 신경망 레이어 등이 정의되어 있음 
import torch.nn.functional as F # Convolution, Pooling, Activation, Linear 함수 등이 정의되어 있음

# torch.nn.Module은 PyTorch의 모든 신경망의 Base Class
# __init()__과 forward()를 반드시 override 해야 한다.
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    # 입력 이미지 채널 1개, 출력 채널 6개, 5x5의 정사각 컨볼루션 행렬
    # 컨볼루션 커널 정의
    # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1)
    self.conv1 = nn.Conv2d(1, 6, 5)   # 입력 채널 크기, 출력 채널 크기, 커널 크기
    self.conv2 = nn.Conv2d(6, 16, 5)  # 입력 채널 크기, 출력 채널 크기, 커널 크기
    
    # Fully Connected Layer: y = Wx + b
    self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5은 이미지 차원에 해당
    self.fc2 = nn.Linear(120, 60)
    self.fc3 = nn.Linear(60, 10)

  def forward(self, x):
    # convolution을 거치게 되면 이미지의 크기는 kernel - 1만큼 감소함
    # 현재 입력되는 이미지의 크기가 32 X 32
    # conv1을 통해 출력되는 이미지의 크기는 (32 - 5 + 1) X (32 - 5 + 1) = 6 X 28 X 28
    # (2, 2) 크기 윈도우에 대해 맥스 풀링 -> 이미지의 크기는 2분의 1이 되므로 -> 최종 출력 이미지 크기는 14 X 14
    # torch.nn.functional.max_pool2d(input, kernel_size)
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # [batch size, 채널 크기, 이미지 가로 크기, 이미지 세로 크기] == [1, 6, 14, 14] 

    # conv2을 통해 출력되는 이미지의 크기는 (14 - 5 + 1) X (14 - 5 + 1) = 10 X 10
    # (2, 2) 크기 윈도우에 대해 맥스 풀링 -> 이미지의 크기는 2분의 1이 되므로 -> 최종 출력 이미지 크기는 5 X 5
    # 크기가 제곱수라면, 하나의 숫자만을 특정(specify)
    x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2)) # [1, 16, 5, 5]
    # torch.flatten(input, start_dim, end_dim) : flattens input by reshaping it into a one-dimensional tensor. 
    x = torch.flatten(x, 1) # batch 차원을 제외한 모든 차원을 하나로 평탄화(flatten) [1, 16 * 5 * 5], Dimension =1 
    x = F.relu(self.fc1(x)) # [1, 120]
    x = F.relu(self.fc2(x)) # [1, 60]
    x = self.fc3(x) # [1, 10]
    return x

net = Net()
print(net)

input = torch.randn(1, 1, 32, 32) # [batch size, 입력 채널 크기, 가로 길이, 세로 길이]
out = net(input)
print("out shape:", out.shape)
print(out)  # out shape: (1, 10)

# view 함수 : reshape 함수. 텐서의 형태(Shape)를 변경함. 변경 전과 후에 원소의 갯수는 유지되어야 한다.
# view(1, -1) : 첫번째 차원은 1이 되도록 하되,-1로 표시된 2번째 차원은 파이토치가 알아서 계산하라는 의미
output = net(input)               # 입력 데이터를 신경망 모델에 전달하여 예측값을 얻음 shape: (1, 10)
print("output shape:", output.shape)
target = torch.randn(10)          # 임의의 텐서를 생성하여 정답값으로 가정
print ("target shape:", target.shape)
target = target.view(1, -1)       # 모델의 출력 텐서와 동일한 shape로 변경 : (1, 10)
print ("after reshape(view) -> target shape:", target.shape)

@ 손실함수 (Loss Function) 설정

import torch
import torch.nn as nn

# 어떤 텐서가 학습에 필요한 텐서라면 backpropagation을 통하여 gradient를 구해야 합니다.
# 텐서의 옵션 requireds_grad를 True로 설정하면 텐서에 실행되는 모든 연산들을 트랙킹하여 자동으로 gradient를 계산합니다.
input = torch.randn(3, 5, requires_grad=True) 
target = torch.randn(3, 5)
 # 손실함수로 MSE(Mean Squared Error)를 설정
 # 평균제곱오차(MSE)는 오차를 제곱한 값의 평균. 오차란 모델이 예측한 값과 실제 정답과의 차이
criterion = nn.MSELoss()          
output = criterion(input, target) # Loss 계산
output.backward()
print('input: ', input)
print('target: ', target)
print('output: ', output)

@ 옵티마이저 설정

# torch.optim : 신경망 학습을 위한 파라미터 최적화 알고리즘이 구현되어 있는 클래스
import torch.optim as optim
# Adam Optimizer 객체 생성
optimizer = optim.Adam(net.parameters(), lr=0.01)

# 학습 과정(training loop)
# Pytorch에서는 gradients 값들을 backward를 할 때 계속 누적하기 때문에 
# 학습을 시작하기 전에 gradients 버퍼를 zero로 reset해야 한다.
optimizer.zero_grad()

input = torch.randn(1, 1, 32, 32)
output = net(input)
target = torch.randn(10)          # 예시를 위한 임의의 정답
target = target.view(1, -1)       # 출력과 같은 shape로 만듬
loss = criterion(output, target)  # Loss 계산
print(loss)

loss.backward()         # 역전파 함수 실행을 통해 gradient 계산
optimizer.step()        # gradient를 기반으로 실제 파라미터 업데이트를 실행

# torch.optim : 신경망 학습을 위한 파라미터 최적화 알고리즘들이 구현되어 있는 팩키지
import torch.optim as optim
# Optimizer 객체를 생성하고 모델의 파라미터를 전달, learning rate 설정
optimizer = optim.Adam(net.parameters(), lr=0.01) 

# 학습 과정(training loop)
# Pytorch에서는 gradients 값들을 backward를 할 때 계속 누적하기 때문에 
# 학습을 시작하기 전에 gradients 버퍼를 zero로 reset해야 한다.
optimizer.zero_grad()

input = torch.randn(1, 1, 32, 32)
output = net(input)
target = torch.randn(10)          # 예시를 위한 임의의 정답
target = target.view(1, -1)       # 출력과 같은 shape로 만듬
loss = criterion(output, target)  # Loss 계산
print(loss)

loss.backward()         # 역전파 함수 실행을 통해 gradient 계산
optimizer.step()        # 파라미터 업데이트를 실행

@ GPU 가속 이용하기
# 현재 개발환경에서 GPU 가속이 가능한지 확인
print(torch.cuda.is_available())

# GPU 사용이 가능하면 cuda로 연산하도록 device를 설정 그렇지 않으면 cpu로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

# 랜덤한 학습 데이터 생성
train_inputs = torch.randn(100, 32, 128) # [전체 데이터 개수, 데이터의 최대 길이, 히든 벡터 크기]
train_labels = torch.randn(100, 3) # [전체 데이터 개수, 예측할 클래스 개수]

# Dataset에 필요한 패키지 임포트
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 12 # 매 학습 Step 마다 샘플링할 데이터 개수

train_data = TensorDataset(train_inputs, train_labels) # 학습데이터와 레이블을 하나의 TensorDataset으로 결합 가능
train_sampler = RandomSampler(train_data) # 데이터를 샘플링 할 함수(순차적으로 뽑아올지, 랜덤하게 뽑아올지)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size) # 미니배치 단위로 데이터를 자동 로딩

# dataloader를 통해서 학습 루프 생성
for step, batch in enumerate(train_dataloader): # enumerate : 인덱스(index)와 데이터값에 동시에 접근
  x_data, x_label = batch
  # 데이터 로딩 step 
  print(step, x_data.shape, x_label.shape)

### Sentiment_CNN

In [ ]:
# torchtext.legacy를 사용할 수 있는 torchtext 버전 설치
!pip install -U torchtext==0.10.0

#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

import torch
# torch.nn : 신경망 구현을 위한 데이터 구조, 신경망 레이어, 관련함수들이 구현되어 있는 팩키지
# torch.nn.functional: torch.nn 팩키지의 함수들이 정의되어 있음 (손실함수, 활성화함수, 풀링함수 등) 
# torch. autograd : 미분을 위한 함수들이 정의되어 있음
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

#import torchtext.data as data
#import torchtext.datasets as datasets
#legacy 버전으로 변경

# torchtext : text의 preprocessing 파이프라인 정의, 
# 토크나이징, Vocab 생성, dataset splits, 데이터 로더 등 지원
from torchtext.legacy import data
import torchtext.datasets as datasets
import pickle

# CNN 모델의 구조와 연산을 정의
class CNN_Text(nn.Module):
    # 생성자 : 모델의 구조와 동작을 정의
    # 객체가 갖는 속성값을 초기화함. 객체가 생성될 때 자동으로 호출된다.
    def __init__(self, embed_num, class_num):
        super(CNN_Text, self).__init__() # nn.Module 클래스를 초기화(부모클래스를 초기화)
        # V: 사전의 크기(vocavilary)
        # D: embed_dim (단어 벡터의 차원)
        # C: 분류하고자 하는 클래스의 개수(긍정 부정 2개 클래스)
        # Co : 각 커널(필터)의 갯수
        V = embed_num
        D = 100 
        C = class_num
        Co = 50         # output channel 수 (필터의 갯수)
        Ks = [2,3,4]

        # 사전에 있는 모든 단어 벡터에 random 초기값
        self.embed = nn.Embedding(V, D) 
        # torch.nn.Conv2d (in_channels, out_channels, kernel_size, stride=1)
        # convs1에 컨볼루션 모듈의 리스트가 들어감 (필터(커널) 갯수만큼) 
        # forward에서 순차적으로 접근 가능
        self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, (K, 100)) for K in Ks])
        self.dropout = nn.Dropout(0.2)

        # nn.Linear 클래스. Fully Connected Layer 
        # Applies a linear transformation to the incoming data (y = Wx + b)
        # torch.nn.Linear(in_features, out_features)
        # in_features: size of each input sample, out_features: size of each output sample 
        self.fc1 = nn.Linear(len(Ks)*Co, C) # 150(3*50), 2 

        # foward 함수 : 모델이 학습데이터를 입력받아서 forward 연산을 진행
        # model 객체를 데이터와 함께 호출하면 자동으로 실행된다.
    def forward(self, x):
        x = self.embed(x)   # (N, W, D) 미니배치, 문장 최대길이, 단어벡터 차원
        x = x.unsqueeze(1)  # (N x Ci x W x D) Conv2d를 사용하려면 입력채널 수 추가해야 함

        # Convolution Layer
        # Convolution -> ReLU -> 텐서의 dimension 3을 squeeze(max_pool1d는 3D 입력을 받음)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]

        # Max Pooling
        # F.max_pool1d(input, kernel_size): Applies a 1D max pooling over an input
        # Tensor.size(dim=None) : Returns the size of the self tensor. If dim is specified, returns the size of that dimension.
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks) max pooling 후에 마지막 차원은 1 -> squeeze
        x = torch.cat(x, 1) # torch.cat(tensors, dim), dim=1이면 두번째 차원이 늘어나게 concat (첫번째 차원은 N)
        x = self.dropout(x) # (N, len(Ks)*Co), dropout을 적용
        logit = self.fc1(x) # fully-connected layer 적용
        return logit
    
class mydataset(data.Dataset):
    @staticmethod  # 유틸리티 메소드 정의 시 선언
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)] # text_field 레이블은 text, label_field 레이블은 label
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:
                    continue
                line = line.strip().split('\t')
                txt = line[1].split(' ')               
                                  
                examples += [ data.Example.fromlist( [ txt, line[2]],fields ) ]
        super(mydataset, self).__init__(examples, fields, **kwargs)
        
# Field : 텐서로 변환될 텍스트 데이터타입을 정의 
# text_field, label_field : 전처리 관련된 field 객체를 각각 생성 
# batch_first : 미니배치 차원을 맨 앞에 둔 텐서를 생성할 것인지
# fix_length : 하나의 문장 내 max 토큰수 
# sequential : 시퀀스 데이터 여부
text_field = data.Field(batch_first = True, fix_length = 20)
label_field = data.Field(sequential= False, batch_first = True, unk_token = None) # unk_token을 표현할 스트링

train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/aivle/data/nsm/small_ratings_train_tok.txt')

test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/aivle/data/nsm/small_ratings_test_tok.txt')
#print(test_data.fields.items())

# vocab 생성
text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

# Data Loader 생성 (train_data, test_data를 각각 100개, 1개씩 데이터 로딩)
train_iter, test_iter = data.Iterator.splits(
                            (train_data, test_data), 
                            batch_sizes=(100, 1))#, device= 'cuda')
len(text_field.vocab)


# CNN모델 객체를 생성 (embed_num, class_num)
cnn = CNN_Text(len(text_field.vocab),2)

# torch.optim : 신경망 학습을 위한 다양한 파라미터 최적화 알고리즘이 구현되어 있는 팩키지
# Optimizer를 설정
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()


for epoch in range(20):
    
    totalloss = 0
    for batch in train_iter:
        optimizer.zero_grad() # resets the gradient to 0
        
        txt = batch.text
        label = batch.label
                
        #print(txt.size()) -> torch.Size([100, 20])
        pred = cnn(txt)
                
        #print(pred.size(), label.size()) -> torch.Size([100, 2]) torch.Size([100])
        #print(label)
        loss = F.cross_entropy(pred, label)
        totalloss += loss.data
        
        loss.backward() # backward 연산
        optimizer.step() # 파라미터 업데이트
        
    print(epoch,'epoch')    
    print('loss : {:.3f}'.format(totalloss.numpy()))

torch.save(cnn,'/content/gdrive/My Drive/aivle/cnn_model.pt')

%%time
from sklearn.metrics import classification_report
cnn.eval() # 모델을 evaluation mode로 설정. 정규화 기술(dropout 등)을 배제하여 온전한 모델로 평가
correct = 0
incorrect = 0
y_test = []
prediction = []

for batch in test_iter:
    txt = batch.text
    label = batch.label
    y_test.append(label.data[0])

    pred = cnn(txt)
    _,ans = torch.max(pred,dim=1) # dimension을 기준으로 (최대값, 최대값이 있는 인덱스) 반환 
    prediction.append(ans.data[0]) # ans.data[0]: 최대값이 들어있는 인덱스 (0 또는 1) 
  
    if ans.data[0] == label.data[0]:        
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print(classification_report(torch.tensor(y_test),     # 정답값
                            torch.tensor(prediction), # 예측값
                            digits=4,                 # 출력할 자리수
                            target_names=['negative', 'positive'])) # display names matching the label

# Weighted Avg는 클래스의 수치간의 평균 

### SonnyBot_CNN

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
    
cd Mecab-ko-for-Google-Colab

!bash install_mecab-ko_on_colab_light_220429.sh 

def mecabsplit(mecab_tagger,inputs, pos):
    r=[]
    inputs = mecab_tagger.parse(inputs)
    t = inputs.split('\n')[:-2]
    for i in t:
        field = i.split('\t')
        if field[1].split(',')[-1] is not '*':
            r.extend( [ (x.split('/')[0],x.split('/')[1]) for x in field[1].split(',')[-1].split('+') ] )
        else:
            r.append( (field[0],field[1].split(',')[0]) )
    if pos:
        return r
    else:
        return [ x[0] for x in r ]
    return r

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable # tensor로 해도됨
import pickle
import MeCab

tagger = MeCab.Tagger()

class CNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num, ):
        super(CNN_Text, self).__init__()
        
        # 단어 사전 크기
        V = embed_num
        # 임베딩벡터 크기
        D = 100 #args.embed_dim
        # 분류하고자 하는 클래스의 개수
        C = class_num
        # 입력 채널 수
        Ci = 1
        # 출력 채널 수
        Co = 20 #args.kernel_num
        # 커널(필터) 사이즈 1단어, 2단어, 3단어
        Ks = [1,2,3]

        self.embed = nn.Embedding(V, D)
        # padding numbers for (height,width)
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D), padding=(2,0)) for K in Ks])
        # dropout 설정
        self.dropout = nn.Dropout(0.2)
        # FC 레이어
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def forward(self, x):
        x = self.embed(x)  # (B, W, D)
        
        # 입력 x를 4D로 변환
        x = x.unsqueeze(1)  # (B(batch), Ci(input channel), W(sent), D(dimension))
        # output = F.relu(x) -> B x Co x W x 1
        # max_pool1D는 3D 입력만 받음 -> size 1인 차원을 제거(squeeze)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(B, Co, W), ...]*len(Ks)
        
        # (B x Co x 1) -> size 1인 차원을 제거(squeeze)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(B, Co), ...]*len(Ks)
        
        # concatenate
        x = torch.cat(x, 1)

        x = self.dropout(x)  # (B, len(Ks)*Co)
        logit = self.fc1(x)  
        return logit
    
# 학습데이터에 나타난 단어 사전
content_vocab = {'unk':0}
# 의도 레이블 사전
intent_vocab={}
# 의도 클래스
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in open('/content/gdrive/My Drive/aivle/data/sonny/mydata.txt','r',encoding='utf-8'):
    line = line.strip().split('\t')
    if len(line)>1:
        intent=line[1]
        if intent not in intent_vocab:
            intent_vocab[intent]=intent_idx
            intent_list.append(intent)
            intent_idx +=1
    else:
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            if it not in content_vocab:
                content_vocab[it] = vocab_idx
                vocab_idx +=1
                
cnn = CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) # 파일에 나타난 단어수, 의도 갯수(Class 갯수)
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()

epoch = 10
for e in range(epoch):
    totalloss = 0
    for line in open('/content/gdrive/My Drive/aivle/data/sonny/mydata.txt','r',encoding='utf-8'):
        line = line.strip().split('\t')
    
        if len(line)> 1:
            target = Variable(torch.LongTensor([intent_vocab[line[1]]]))
            continue

        optimizer.zero_grad()

        cont = []
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            cont.append(content_vocab[it])
        # view : 원소의 수를 유지하면서 텐서를 reshape, 텐서의 첫번째 차원을 1로 reshape
        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)

        loss = F.cross_entropy(pred,target)
        totalloss += loss.data
        loss.backward()
        optimizer.step()
    print (e, 'epoch')
    print('loss : {:.3f}'.format(totalloss.numpy()))
    
response = []
for line in open('/content/gdrive/My Drive/aivle/data/sonny/response.txt','r',encoding='utf-8'):
    line=line.strip()
    response.append(line)
    
    
cnn.eval()
for line in open('/content/gdrive/My Drive/aivle/data/sonny/testdata.txt','r',encoding='utf-8'):
    line = line.strip()
    
    line = mecabsplit(tagger,line,False)
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it]) # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = cnn(cont)
    v,i = torch.max(pred,1) # pred는 (p1, p2) 즉, 클래스별 확률 v: 둘중 큰값 i:큰값 클래스의 인덱스
    
    print('input : ',line)
    # 3개 클래스의 확률값
    probs = torch.nn.functional.softmax(pred,dim=-1).data.numpy()[0]
    print ([probs[0], probs[1], probs[2]])
    print('intent : ',intent_list[int(i)])
    print(response[int(i)])
    print()

### Sentiment_RNN

In [ ]:
# torchtext.legacy를 사용할 수 있는 torchtext 버전 설치
!pip install -U torchtext==0.10.0

#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

import torch
import torch.nn as nn # 뉴럴 네트워크
import torch.nn.functional as F

# torchtext.legacy : text의 preprocessing 파이프라인 정의
# 1) 토크나이징(Tokenization)
# 2) 단어장 생성(Build Vocabulary)
# 3) 토큰의 수치화(Numericalize all tokens)
# 4) 데이터 로더 생성(Create Data Loader)
from torchtext.legacy import data
import torchtext.datasets as datasets

import pickle
print (torch.__version__)

class RNN_Text(nn.Module):   # init, forward를 꼭 재정의 해줘야함 
    def __init__(self, embed_num, class_num):
        # super()로 Base Class의 __init__() 호출 (nn.Module 클래스 생성자 호출)
        # super(파생클래스, self).__init__() 파이썬 2.x 문법
        # super().__init__() 파이썬 3.x 문법 둘다 사용 가능
        super(RNN_Text, self).__init__()
          
        V = embed_num   # 단어 사전의 크기
        C = class_num   # 분류하고자 하는 클래스 개수        
        H = 256         # 히든 사이즈
        D = 100         # 단어벡터 차원 100        
        self.embed = nn.Embedding(V, D)        
        
        # LSTM Layer, bidirectional이므로 출력되는 벡터의 크기는 H * 2
        self.rnn = nn.LSTM(D, H, bidirectional = True) # bidirectional = 양방향의 여부
                 
        # Linear Layer : (512, 2) # 긍정부정(2개)
        self.out = nn.Linear(H*2, C) # 마지막 상태의 히든스테이트가 됨(양방향이 되기 때문에 256*2)
        
    def forward(self, x):
        x = self.embed(x)     # (N, W, D) 문장 x의 단어 벡터값 가져옴 (임베딩해줌) 차원을 나타냄, 한문장과 batch size를 곱해줌
      
        # LSTM 모듈 실행
        # LSTM 입력데이터
        # input x : torch.Size([30, 100, 100]) [시퀀스 길이, 배치 사이즈, Dimension] -> 문장하나다
        x,(_,__) = self.rnn( x, ( self.h, self.c ) )  # 몇차원의 텐서가 들어가고 나오는지가 중요함

        # output x : torch.Size([30, 100, 512]) [시퀀스 길이, 배치 사이즈, 256 * 2]# 마지막 상태의 히든스테이트가 됨
        # print('output * size', x.size)
        # 최종 Hidden Layer로 Linear 모듈 실행   
        logit = self.out(x[-1])  # 첫번째 차원, 즉 시퀀스차원에 있는 마지막 단어의 배치값이 담겨있는 tensor를 가져옴

        # 최종 예측 벡터 크기: [배치 사이즈, C], C: 클래스 개수
        return logit       # logit : torch.Size([100, 2])

    def inithidden(self, b): # 히든스테이트 초기화 함수(cell, hidden 둘다 랜덤으로 초기화)
        #self.h = Variable(torch.randn(2, b, 256))
        #self.c = Variable(torch.randn(2, b, 256))    
        self.h = torch.randn(2, b, 256)   # [2, batch_size, 256]
        self.c = torch.randn(2, b, 256)   # [2, batch_size, 256]
        
        
# train, test dataset을 만들어준다
class mydataset(data.Dataset):
    @staticmethod
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:      # 첫번째 라인은 skip
                    continue
                line = line.strip().split('\t') # text, label 필드가 /tab으로 구분되어 있다                  
                txt = line[1].split(' ')  # 공백을 기준으로 문자열을 나누어 토큰 리스트를 만든다. line[0]에는 ID
               
                # examples: 학습 텍스트, 라벨 텍스트
                # data.Example : Defines a single training or test example.
                examples += [ data.Example.fromlist( [txt, line[2]],fields ) ]
        # Create a dataset from a list of Examples and Fields.
        # fields : field name, field 
        super(mydataset, self).__init__(examples, fields, **kwargs) 

        
# Field 객체는 다음과 같은 값을 통하여 데이터의 각 필드를 처리하는 방법을 지정
# fix_length: A fixed length that all examples using this field will be padded to, or None for flexible sequence lengths. 
# sequential: Whether the datatype represents sequential data. If False, no tokenization is applied. Default: True.
# batch_first: Whether to produce tensors with the batch dimension first. Default: False.
##text_field = data.Field(fix_length=20)
text_field = data.Field(fix_length=30)
label_field = data.Field(sequential=False, batch_first = True, unk_token = None)

# 학습데이터 Dataset
train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/aivle/data/nsm/small_ratings_train_tok.txt')
# 테스트데이터 Dataset
test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/aivle/data/nsm/small_ratings_test_tok.txt')

text_field.build_vocab(train_data)    # Construct the Vocab object 
label_field.build_vocab(train_data)   # Construct the Vocab object 

# Create Iterator objects for train data, test data
train_iter, test_iter = data.Iterator.splits(
                            (train_data, test_data), 
                            batch_sizes=(100, 1), repeat=False)#, device = -1)
len(text_field.vocab)

rnn = RNN_Text(len(text_field.vocab),2)     # embed_num, class_num
optimizer = torch.optim.Adam(rnn.parameters())
rnn.train()

%%time
bool_debug = True    # 텐서의 차원을 출력할 경우 True로 설정
print_idx = 3        # 출력 횟수
for epoch in range(10):
    
    totalloss = 0
    for batch in train_iter:
        optimizer.zero_grad()
        
        txt = batch.text        # torch.Size([30, 100])
        label = batch.label     # torch.Size([100])
        
        if bool_debug and print_idx > 0:
          # print('txt.size():', txt.shape)
          print ("txt.shape:", txt.shape)
          print_idx -= 1

        # inithiddend : hidden state, cell state 초기화 함수
        rnn.inithidden(txt.size(1))   # 배치 사이즈를 전달
        # 학습 실행
        pred = rnn(txt)
        
        if bool_debug and print_idx > 0:
          print("pred.shape:", pred.shape)
          print("label.shape:", label.shape)
          print_idx -= 1        

        loss = F.cross_entropy(pred, label)
        totalloss += loss.data
        
        loss.backward()
        optimizer.step()
        
    print(epoch,'epoch')  
    print('loss : {:.3f}'.format(totalloss.numpy()))
       
torch.save(rnn,'/content/gdrive/My Drive/aivle/model/rnn_model.pt')

%%time
bool_debug = True    # 텐서의 차원을 출력할 경우 True로 설정

from sklearn.metrics import classification_report
correct = 0
incorrect = 0
rnn.eval()
y_test = []
prediction = []

# 텐서 차원 확인용
print_tensor_shape = 2
print_idx = 1

for batch in test_iter:
    txt = batch.text            # txt.shape: torch.Size([max_sent_len, 1])
    label = batch.label         # label.shape: torch.Size([1])
    y_test.append(label.data[0])
    
    rnn.inithidden(txt.size(1))
   
    pred = rnn(txt)               # pred.shape: torch.Size([1, 2])
    
    _ , ans = torch.max(pred,dim=1) # ans.shape: torch.Size([1])
    prediction.append(ans.data[0])
    
    
    #---------------------------------------
    # 텐서 형태, 데이터를 출력
    if bool_debug and print_tensor_shape > 0:
      print("-----", print_idx, "-----") 
      print("prediction:", prediction)
      print("y_test:", y_test)
      print("pred.shape:", pred.shape)
      #print("pred.data[0]:", pred.data[0])
      print("pred[0]:", pred[0])
      print("pred[0][0]:", pred[0][0])
      print("pred[0][1]:", pred[0][1])
      print("ans.data[0]:", ans.data[0])
      print("ans.shape:", ans.shape)
      print("txt.shape:", txt.shape)
      print("label.shape:", label.shape)
      print("label.data[0]:", label.data[0])
      
      print()
      print_tensor_shape -= 1
      print_idx += 1
      #---------------------------------------

    if ans.data[0] == label.data[0]:  # ans.data[0]: tensor(0) 또는 tensor(1)
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print(classification_report(torch.tensor(y_test), 
                            torch.tensor(prediction), 
                            digits=4, 
                            target_names=['negative', 'positive']))

### Intent_analysis_CNN

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh 

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable 
import pickle 

import MeCab
tagger = MeCab.Tagger()
#local에서 실행할 경우 참고사항 : 사전 디렉토리를 인자로 전달할 수 있음
#tagger = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')

from google.colab import drive
drive.mount('/content/gdrive')

df=pd.read_csv('/content/gdrive/My Drive/aivle/data/college/college_FAQ.csv',encoding='utf-8', on_bad_lines='skip')
df.tail()

df.category.value_counts()

subset_category=['등록','휴복학','수강신청','장학금','생활관']

df=df[df.category.isin(subset_category)]
# 학습데이터 원문 확인
print (df.head())
print()
print (df.tail())
print()

def mecabsplit(mecab_tagger,inputs, pos):
    r=[]
    inputs = mecab_tagger.parse(inputs)
    t = inputs.split('\n')[:-2]
    for i in t:
        field = i.split('\t')
        if field[1].split(',')[-1].strip() is not '*':
            r.extend( [ (x.split('/')[0],x.split('/')[1]) for x in field[1].split(',')[-1].split('+') ] )
        else:
            r.append( (field[0],field[1].split(',')[0]) )
    if pos:
        return r
    else:
        return [ x[0] for x in r ]
    return r

content_vocab = {'unk':0}
intent_vocab={}
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in df.category.unique():
    intent=line
    if intent not in intent_vocab:
        intent_vocab[intent]=intent_idx
        intent_list.append(intent)
        intent_idx +=1

for line in list(df.question):
    line = mecabsplit(tagger,line,False)
    for it in line:
        if it not in content_vocab:
            content_vocab[it] = vocab_idx # it : 단어
            vocab_idx +=1
            
print("vocab size:", len(content_vocab))
print(content_vocab)  # 단어 사전
print(intent_vocab)   # intent 사전
print("intent size:", len(intent_list))
print(intent_list)    # intent_list : FAQ 카테고리 리스트

@ 모델링
class CNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num, ):
        super(CNN_Text, self).__init__()

        #--- <연습2> 아래 빈 코드를 채우세요
        V = embed_num
        D = 100            # 단어임베딩 차원 
        C = class_num   # 분류할 카테고리 갯수
        Ci = 1          # 입력 채널의 수
        Co = 20           # 출력 채널의 수
        Ks = [1,2,3]           # 커널(필터) 사이즈 정의
        #--- end

        self.embed = nn.Embedding(V, D)  # 단어 벡터를 임의의 값으로 초기화

        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])

        self.dropout = nn.Dropout(0.2)  # dropout 정의
        
        #--- <연습3> Fully-connected layer를 정의하세요
        self.fc1 = nn.Linear(len(Ks)*Co, C)                     
        #--- end

    def forward(self, x):
        # x : 학습데이터 파일의 한 문장
        # 입력문장이 [대학, 원, 수료, 생, 논문, 제출] => 
        # x: [1, 2, 3, 4, 8, 9]의 입력이 생성됨 (content_vocab 출력결과 참고)
        
        # (N, W) Integer => (W) => view(1, -1) => (1, W)
        x = self.embed(x)         # (N, W, D) (1문장, 최대 입력 단어 개수, D차원)
        # 문장 x의 단어 벡터값 가져옴
        #print (x.size())

        #--- <연습4> unsqueeze 함수를 호출하세요
        x = x.unsqueeze(1)      # (N, Ci, W, D) -> Ci를 추가하여 unsqueeze (Text는 채널 1개, 이미지는 채널 3개)
        #--- end
        
        #print (x.size())
        
        # 컨볼루션 부분 : 입력문 별로 호출되어 커널을 적용하면서 컨볼루션 수행
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)
        
        #print(len(x))
        #print(x[0].size())
        #print(x[1].size())
        #print(x[2].size())
        
        # max pooling 부분. max pooling을 수행한 결과 벡터의 차원은? (=출력채널의 갯수)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
                
        x = torch.cat(x, 1)

        x = self.dropout(x)  # (N, len(Ks)*Co)
        
        # 결과값(logit)은 실수값 2개를 출력하며 이 중 큰값을 가진 쪽이 예측값
        logit = self.fc1(x)  # (N, C) 

        #print(logit)
        return logit

    
cnn = CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) # 파일에 나타난 단어수, 의도 갯수(클래스)
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()    # train mode로 준비

@ Training
epoch = 10
for e in range(epoch):
    totalloss = 0
    
    for line in df.values:
        target=Variable(torch.LongTensor([intent_vocab[line[1]]])) # 딕셔너리 타입

        optimizer.zero_grad()

        cont = []
        # 입력 문장을 형태소 단위의 토큰으로 분리
        line = mecabsplit(tagger,line[2],False)
        
        for it in line:
            # 형태소 토큰을 content_vocab을 기준으로 id 값으로 변환하여 입력 생성
            cont.append(content_vocab[it])
        
        # list 변수를 torch에서 사용가능한 data_type으로 변환
        # view 함수는 입력 텐서를 reshape하는 함수
        # [입력 토큰 개수] => [1, 입력 토큰 개수]
        # 
        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)
        
        #--- <연습5> 정답값과 예측값의 오차를 계산하는 손실함수 cross_entropy를 호출하세요
        loss = F.cross_entropy(pred,target)
        #--- end

        totalloss += float(loss.data)
        loss.backward()  # 필터, 임베딩벡터 값, FC 레이어의 matrix 등 모든 값을 loss를 줄이는 방향으로 계산

        
        #--- <연습6> 파라미터값을 변경하는 함수를 호출하세요
        # write code here
        optimizer.step()
        #--- end
    
    print(e,'epoch')    
    print('loss : {:.3f}'.format(totalloss))
    #print (totalloss)

    
@ Response
response = df.answer
with open('/content/gdrive/My Drive/aivle/data/college/college_FAQ_answer.txt', 'r', encoding='utf-8') as response_file:
    response = [row.split('\t')[1] for row in response_file.readlines()[1:]]
    
@ Test
test=[]
# intent_list는 FAQ 카테고리 리스트:
# ['등록', '휴복학', '수강신청', '장학금', '생활관']
for i in intent_list:
    subset_df=df.query('category=="{}"'.format(i))
    test.append(subset_df.question.values[0])
    print(subset_df.question.values[0])
    
test

# your test : input questions
#test=['2학기 수강신청 기간 언제인가요?']
#test=['장학금 받으려면 몇 학점 이상이어야 되나요?']
test = ['재수강 들어가려면 어떻게?']

for idx in list(test):
    line = mecabsplit(tagger,idx, False)
   
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it])  # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = cnn(cont)
    #pred : (1, C)
    v,i = torch.max(pred,1) # i: 가장 큰 값이 들어있는 위치 인덱스
    
    print('input : ', line)
    print('intent : ', intent_list[int(i)])
    #print(int(i))
    print('answer : ', response[int(i)],'\n')

### bert_for_classification

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

@ Import modules
import pickle as pc
import os
import numpy as np
# csv 모듈은 CSV 형식(쉼표로 구분된 표 형식) 데이터를 읽고 쓰는 클래스를 제공
import csv
import torch
# torch 버전 확인
print("Pytorch Version: ", torch.__version__)
# GPU 사용 가능한지 여부 확인
if torch.cuda.is_available():
    # PyTorch 에게 GPU 사용할거라고 알려주기
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

@ Installing the Hugging Face Library
# transformers 팩키지 설치
!pip install transformers

@ Configure the experiments
train_filename = '/content/gdrive/My Drive/aivle/data/amazon/bert_train_data_all.csv'
test_data = '/content/gdrive/My Drive/aivle/data/amazon/bert_balanced_data'
test_label = '/content/gdrive/My Drive/aivle/data/amazon/bert_balanced_label'

@ Load Amazon Review Dataset
def load_data(filename):
    data = list()
    label = list()
    
    f = open(filename, 'r', encoding='utf-8')
    reader = csv.reader(f)
    for idx, line in enumerate(reader):
        if idx == 0:
            continue
        # line[2]에 label 1(긍정), 0(부정) line[5]에 review text가 있음 
        data.append(line[5])
        label.append(int(line[2]))

    f.close() 
   
    # data와 label 사이즈 일치 여부 확인
    assert len(data) == len(label)  
    return data, label

# train_data : 리뷰문장 (text), train_lable : 1(긍정) 또는 0(부정)
train_data, train_label = load_data(train_filename)

print("Size of train data: {}".format(len(train_data)))
print("Size of train label: {}".format(len(train_label)))

@ Tokenization & Input Formatting
## BERT Tokenizer
from transformers import BertTokenizer

# BERT tokenizer 불러오기
# do_lower_case : True이면 모두 소문자로 변환, False이면 대소문자 구분
print("Loading BERT tokenizer...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# 하나의 sentence에 대해 BertTokenizer 적용
# Print the original sentence.
print("Original: ", train_data[0])
print("Original: ", train_data[1])
print()
# Print the sentence split into tokens.
print("Tokenized: ", tokenizer.tokenize(train_data[0]))
print("Tokenized: ", tokenizer.tokenize(train_data[1]))
print()
# Print the sentence mapped to token ids.
print("Token IDs: ", tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_data[0])))
print("Token IDs: ", tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_data[1])))
print()

## Required Formatting
각 문장의 처음과 끝에 special token 더하기
각 문장을 maximum length 만큼 자르고 padding token 채워주기
각 문장에서 padding token 과 실제 token 들 구분하기 위한 attention masking 적용

# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []

# For every sentence
for sent in train_data:
    # 'encode' will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the '[CLS]' token to the start.
    #   (3) Append the '[SEP]' token to the end.
    #   (4) Map tokens to their IDs.
    #   max_length : 문장의 최대길이
    #   encoded_sent : token IDs
    #---------------------------------------------------------------
    #      연습 (1)     tokenizer의 encode 함수를 호출해 주세요.  
    #---------------------------------------------------------------                
  
    encoded_sent = tokenizer.encode(sent, 
                                    add_special_tokens=True,
                                    max_length = 64)
    
    # Add the encoded sentence to the list
    input_ids.append(encoded_sent)

# Print train data[0]
print("Original: ", train_data[0])
print()
print("Token IDs: ", input_ids[0])

# Print special tokens and tokenized sentence
print("\n[CLS] token: {:}, ID: {:}".format(tokenizer.cls_token, tokenizer.cls_token_id))
print("\n[PAD] token: {:}, ID: {:}".format(tokenizer.pad_token, tokenizer.pad_token_id))
print("\n[SEP] token: {:}, ID: {:}".format(tokenizer.sep_token, tokenizer.sep_token_id))
print("\nTokenized: ", tokenizer.convert_ids_to_tokens(input_ids[0]))

# Padding & Truncating
print("Max length: ", max([len(each) for each in input_ids]))
import tensorflow as tf
print("Tensorflow version: {}".format(tf.__version__))

MAXLEN = 64
# post-sequence truncation, post-sequence padding
# padding value 0, type of output sequences long
input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, 
                                                          maxlen=MAXLEN, dtype="long", value=0, truncating="post", padding="post")

print("\nPadding is done.")

# Attention Masks
# Create attention masks
attention_masks = [] # 각 문장에 대한 attention mask 리스트를 저장

# 토큰 시퀀스에서 패딩에 해당하는 부분은 0, 패딩이 아닌 부분은 1을 넣은 mask를 생성
# 패딩 부분은 모델 내에서 Attention을 수행하지 않아 학습속도를 향상
# For every sentence
for sent in input_ids:
    # Create the attention mask.
    #  - If a token ID is 0, then it's padding, set the mask to 0.
    #  - If a token ID is not 0 ( > 0), then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)
print("\nAttention masking is done.")

# Training & Validation Split
# Use train_test_split to split our data into train and validation sets for training
from sklearn.model_selection import train_test_split
# train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)
#     arrays : 분할시킬 데이터
#     test_size : 테스트 데이터셋의 비율 (default = 0.25)
#     random_state : 데이터 셔플 시 seed value. 호출할 때마다 동일한 학습 데이터, 테스트 데이터 셋을 생성하기 위해 설정
#     shuffle : 셔플 여부 (default = True)
#     stratify : 지정한 Data의 비율을 유지한다. 예를 들어, Label Set인 Y가 25%의 0과 75%의 1로 이루어진 Binary Set일 때
#     stratify=Y로 설정하면 나누어진 데이터셋들도 0과 1을 각각 25%, 75%로 유지
# 반환값 : (학습 데이터, 테스트 데이터, 학습데이터 label, 테스트데이터 label)
# test_size=0.1로 설정, Use 90% for training and 10% for validation
train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(input_ids, train_label, random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, valid_masks, _, _ = train_test_split(attention_masks, train_label, random_state=2018, test_size=0.1)
# print train_inputs, valid_inputs
# 첫번째 학습데이터, 첫번째 attention mask를 출력해서 확인
print(train_inputs[:1])
print(train_masks[:1])

# Converting to PyTorch Data Types
# Convert all inputs and labels into torch tensors, the required data type for our model.
# torch.tensor(data) -> tensor를 생성
train_inputs = torch.tensor(train_inputs)
valid_inputs = torch.tensor(valid_inputs)

train_labels = torch.tensor(train_labels)
valid_labels = torch.tensor(valid_labels)

train_masks = torch.tensor(train_masks)
valid_masks = torch.tensor(valid_masks)

# PyTorch 의 DataLoader class 를 이용하여 amazon review dataset 에 대한 iterator 를 생성합니다.
# torch.utils.data : 파이토치의 데이터 로딩 유틸리티. 데이터셋, 데이터로더, 샘플러 등을 제공함
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# The DataLoader needs to know our batch size for training, so we specify it here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.
batch_size = 32
#---------------------------------------------------------------
#      연습 (2) train_data, valid_data의 데이터 로더를 생성해 주세요.
#---------------------------------------------------------------   
# Create the DataLoader for our training set.
# torch.utils.data.TensorDataset(*tensors)
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
valid_data = TensorDataset(valid_inputs, valid_masks, valid_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

@ Train our classification model
## BertForSequenceClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassfication, the pretrained BERT model 
# with a single linear classification layer on top.
# BERT 모델의 네트워크 형태를 출력
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab
                                                      num_labels = 2, # The number of output labels 2 for binary classification
                                                                      # You can increase this for multi-class tasks
                                                      output_attentions = False, # whether the model returns attentions weight (correponding to multi-head self attentions)
                                                      output_hidden_states = False) # whether the model returns all hidden states

# Tell PyTorch to run this model on the GPU
# (model의 모든 parameter를 GPU에 loading)
model.cuda()

# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print("The BERT model has {:} different named parameters.\n".format(len(params)))
print("=== Embedding Layer ===\n")
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("\n==== First Transformer ====\n")
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("\n==== Output Layer====\n")
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
    
## Optimizer & Learning Rate Scheduler
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for "Weight Decay fix"
# Weight Decay: weight들의 값이 증가하는 것을 제한함으로써 모델의 복잡도를 감소시켜 오버피팅을 방지하는 기법 
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

from transformers import get_linear_schedule_with_warmup
# Number of training epochs (we recommend between 2 and 4)
epochs = 4
# Total number of training steps is number of batches * number of epochs.
print("number of batches:", len(train_dataloader))
total_steps = len(train_dataloader) * epochs
print("total_steps:", total_steps)
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

## Training Loop
# Function to calculate the accuracy of our predictions vs labels
# numpy argmax : 해당 차원(axis)의 값 중에서 가장 큰 값의 인덱스를 반환
# flatten() : 다차원 배열을 1차원으로 변환
# sum() : 배열 내 전체 값들의 합
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
import time
import datetime
def format_time(elapsed):
    '''
    Take a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round(elapsed))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
import random
# Set the seed value all over the place to make this reproducible
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
seed_val = 42
set_seed(seed_val)
# Store the average loss after each epoch so we can plot them
loss_values = []
# For each epoch
for epoch in range(0, epochs):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0.
    # Put the model into training mode.
    # Don't be mislead -- the call to 'train' just changes the "mode", it doesn't "perform" the training.
    # 'dropout' and 'bachnorm' layers behave differently during training vs test
    model.train()
    # For each batch of training data
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress
            print("Batch {:>5,} of {:>5,}. Elapsed: {:}.".format(step, len(train_dataloader), elapsed))
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        #---------------------------------------------------------------
        #      연습 (3) 아래 라인에 그래디언트를 0으로 초기화하는 함수를 호출해 주세요.
        #--------------------------------------------------------------- 
        model.zero_grad()
        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value from the tensor.
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.        
        #---------------------------------------------------------------
        #      연습 (4) 아래 라인에 파라미터를 업데이트하는 함수를 호출해 주세요.
        #--------------------------------------------------------------- 
        optimizer.step() # 최적화, 업데이트 
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("Average training loss: {0:.2f}".format(avg_train_loss))
    print("Training epoch took: {:}".format(format_time(time.time() - t0)))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode -- the dropout layers behave differently during evaluation
    model.eval()
    # Tracking variables
    eval_loss, eval_acc = 0., 0.
    # Evaluate data for one epoch
    for valid_step, batch in enumerate(valid_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            #---------------------------------------------------------------
            #      연습 (5) 아래 함수에서 파라미터를 채워주세요
            #--------------------------------------------------------------- 
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        # Get the "logits" output by the model.
        # The "logits" are the output values prior to applying an activation function like the softmax
        # output 타입: class transformers.modeling_outputs.SequenceClassifierOutput
        # (https://huggingface.co/docs/transformers/v4.22.2/en/main_classes/output#transformers.modeling_outputs.SequenceClassifierOutput)
        # our model will return (outputs.loss(optional)=None, outputs.logits)
        logits = outputs[0]
        # Move logits and labels to CPU (Module을 통해 나온 tensor을 후처리에 사용하거나, 계산된 loss를 로깅 등)
        # detach() : 파이토치는 tensor에서 이루어진 모든 연산을 추적해서 graph에 기록해두는데 이 연산 기록으로부터 
        # 그래디언트가 계산되고 역전파가 이루어지게 된다. detach()는 이 연산 기록으로부터 분리한 tensor을 반환하는 method     
        # cpu() : GPU 메모리에 올려져 있는 tensor를 cpu 메모리로 복사하는 method   
        # numpy() : tensor를 numpy로 변환하여 반환
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_acc = flat_accuracy(logits, label_ids)
        # Accumulate the total accuracy.
        eval_acc += tmp_eval_acc        
    # Report the final accuracy for this validation run.
    print("Accuracy: {0:.2f}".format(eval_acc / (valid_step + 1)))
    print("Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")        
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
# Use plot styling from seaborn.
sns.set(style='darkgrid')
# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)
# Plot the learning curve.
plt.plot(loss_values, 'b-o')
# Label the plot.
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

@ Performance on Test set
## Load Amazon Review Test Dataset
# pickle 모듈로 테스트 파일을 load : bert_balanced_data, bert_balanced_label
with open(test_data, 'rb') as f:
    test_data = pc.load(f)
with open(test_label, 'rb') as f:
    test_label = pc.load(f)
print("Size of test data: {}".format(len(test_data)))
print("Size of test label: {}".format(len(test_label)))

## Tokenization & Input Formatting
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in test_data:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    input_ids.append(encoded_sent)
# Pad our input tokens
input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=MAXLEN, 
                          dtype="long", truncating="post", padding="post")
# Create attention masks array
attention_masks = []
# For every sentence
for sent in input_ids:
    # Create the attention mask.
    #  - If a token ID is 0, then it's padding, set the mask to 0.
    #  - If a token ID is not 0 ( > 0), then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)
# Convert to tensors
# input_ids : 테스트 문장에 포함된 단어 ids
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(test_label)
# Set the batch size.  
batch_size = 32  
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
# For validation the order doesn't matter, so we'll just read them sequentially.
prediction_sampler = SequentialSampler(prediction_data) 
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

## Evaluate on Test Set
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions, true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader (batch에서 데이터 추출)
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  # logist, labels 텐서를 CPU로 이동하여 정확도 계산
  # detach() : 텐서에서 이루어진 연산 기록으로부터 분리한 텐서를 반환
  # cpu() : GPU 메모리에 올려져 있는 tensor를 cpu 메모리로 복사
  # numpy() : tensor를 numpy로 변환하여 반환
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  # Store predictions and true labels
  # predictions : 예측값 array
  # true_labels : 정답값 array
  # logits array의 값들 중 가장 큰 값의 index를 반환
  # when axis = 1, argmax identifies the maximum value for every row. 
  predictions.extend(np.argmax(logits, axis=1).flatten()) # [0, 1, 1, 0, ...]
  true_labels.extend(label_ids.flatten()) # [0, 1, 0, 0, ...]
print('DONE.')

# accuracy, precision, recall, f1 score 성능 확인
from sklearn.metrics import classification_report
target_names = ['negative', 'positive']
# sklearn.metrics.classification_report(y_true, y_pred, digits, target_names)
# support is the number of actual occurrences of the class in the specified dataset.
print(classification_report(true_labels, predictions, digits=4, target_names=target_names))

## 스팸분류

In [ ]:
!pip install rich

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import rich  # 출력을 예쁘게 꾸며주는 라이브러리
from rich.table import Table

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from tqdm.auto import tqdm

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

train_df = pd.read_csv('spam.csv')
test_df = pd.read_csv('spam_test_text.csv')

In [ ]:
train_df.dropna(inplace=True)
train_df.isna().sum()
train_df.reset_index(inplace=True)
train_df.drop(columns={'index'},inplace=True)
train_df

In [ ]:
train_df.dropna(inplace=True)
train_df['label'] = train_df['label'].replace(['ham','spam'],[0,1])

In [ ]:
def get_pipe(model, model_name: str) -> Pipeline:
    "TfidfVectorizer와 모델을 연결한 파이프라인을 반환하는 함수"
    tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1, 3))
    pipe = Pipeline([
        ("tfidf", tfidf),
        (model_name, model)
    ])
    return pipe

In [ ]:
def return_kfold_accuarcy(model, k: int = 5) -> float:
    "모델을 입력받아 KFold 예측 후 accuracy score를 반환하는 함수"
    kfold = StratifiedKFold(k, shuffle=True, random_state=42)
    result = []
    for train_idx, test_idx in kfold.split(train_df["text"], train_df["label"]):
        train, val = train_df.iloc[train_idx], train_df.iloc[test_idx]
        model.fit(train["text"], train["label"])
        pred = model.predict(val["text"])
        acc = accuracy_score(val["label"], pred)
        result.append(acc)

    return np.mean(result)

In [ ]:
models = [
    ("naive_bayes", BernoulliNB()),
    ("SGD", SGDClassifier(random_state=42, n_jobs=-1)),
]

model_pipes = [(name, get_pipe(model, name)) for name, model in models]

In [ ]:
# models = [
#     ("naive_bayes", BernoulliNB()),
#     ("SGD", SGDClassifier(random_state=42, n_jobs=-1)),
#     ("rfc", RandomForestClassifier(random_state=42, n_jobs=-1)),
#     ("SVC", SVC(random_state=42)),
#     ("ada", AdaBoostClassifier(random_state=42)),
#     ("lgbm", LGBMClassifier(random_state=42)),
#     ("lgbm2", LGBMClassifier(n_estimators=80, random_state=42)),
#     ("xgb", XGBClassifier(random_state=42)),
#     ("knc1", KNeighborsClassifier()),
#     ("knc2", KNeighborsClassifier(n_neighbors=4))
# ]

# model_pipes = [(name, get_pipe(model, name)) for name, model in models]

In [ ]:
table = Table(title="Model Comparison Table")
table.add_column("Model Name", justify="left", style="green")
table.add_column("Accuracy", justify="right")

for model_name, model in tqdm(model_pipes, leave=False):
    acc = return_kfold_accuarcy(model)
    table.add_row(model_name, f"{acc:0.3f}")

rich.print(table)

In [ ]:
from sklearn.ensemble import StackingClassifier

stack_models = [(name, get_pipe(model, name)) for name, model in models]

stacking = StackingClassifier(stack_models)
acc = return_kfold_accuarcy(stacking)
rich.print(acc)

In [ ]:
stacking.fit(train_df['text'], train_df['label'])
submission_pred = stacking.predict(test_df['text'])

In [ ]:
submission = pd.read_csv('spam_submission.csv')

In [ ]:
submission['label'] = submission_pred
submission

In [ ]:
submission['label']= submission['label'].replace([0,1],['ham','spam'])

In [ ]:
submission.to_csv("/aihub/data/M1.csv",index=False)

# 수어번역[CNN]

In [ ]:
# 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
import cv2, os

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.backend import clear_session
from tensorflow.keras.optimizers import Adam

In [ ]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err')
    plt.plot(history['val_loss'], label='val_err')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '내경로'

data = pd.read_csv(path)
data.head()
data.shape

In [ ]:
# class name 찍어보기
import string
class_names = list(string.ascii_lowercase)
len(class_names), class_names


# 데이터 살펴보기
# 아래 숫자를 바꿔가며 화면에 그려 봅시다.
n = 10
sign_fig = data.iloc[n, 1:].values
sign_fig = sign_fig.reshape(28, 28)

sign = class_names[data.iloc[n,0]]

plt.title(sign)
plt.imshow(255-sign_fig, cmap=plt.cm.binary)
plt.colorbar()
plt.show()

In [ ]:
# 전처리
target = 'label'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

x_train, x_val, y_train, y_val = train_test_split(x, y, train_size = 20000, random_state = 2022)

x_train.shape, x_val.shape

# 모두 넘파이로 변환
x_train2, x_val2, y_train2, y_val2 = x_train.values, x_val.values, y_train.values, y_val.values


## CNN을 위해 shape 맞추기 n, 28,28,1
x_train2 = x_train2.reshape(20000,28,28,1)
x_val2 = x_val2.reshape(7455,28,28,1)

x_train2.shape, x_val2.shape

# min_max scaling
x_train2 = x_train2 / 255.
x_val2 = x_val2 / 255.

In [ ]:
# 모델링(요즘 뜨는거-> CatBoost)
from sklearn.ensemble import RandomForestClassifier

m1 = RandomForestClassifier()
m1.fit(x_train, y_train)
p1 = m1.predict(x_val)

cn = np.array(class_names)

print(accuracy_score(y_val,p1))
print('-'*60)
print(confusion_matrix(y_val, p1))
print('-'*60)
print(classification_report(cn[y_val], cn[p1]))

## CNN
clear_session()

m2 = Sequential([Conv2D(32, kernel_size=3, input_shape=(28, 28, 1), padding='same', strides =1, activation='relu'),
                    MaxPooling2D(pool_size=2, strides=2),
                    Flatten(),
                    Dense(128, activation = 'relu'),
                    Dense(25, activation='softmax')
])

m2.summary()

m2.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy')

history = m2.fit(x_train2, y_train2, epochs = 10, validation_split=0.2).history

dl_history_plot(history)

In [ ]:
# 예측 & 검증
p2 = m2.predict(x_val2)

p2_1 = p2.argmax(axis=1)

cn = np.array(class_names)

print(accuracy_score(y_val,p2_1))
print('-'*60)
print(confusion_matrix(y_val, p2_1))
print('-'*60)
print(classification_report(cn[y_val], cn[p2_1]))

In [ ]:
# 모델 저장하기(3가지 방식)keras, joblib, pickle
import joblib
joblib.dump(m1, 'sign_model_rf.pkl')
m2.save('sign_model.h5')

# 모델 로딩
m1_2 = joblib.load('sign_model_rf.pkl')
from keras.models import load_model
m2_1 = load_model('sign_model.h5')

# 모델 사용
p1_2 = m1_2.predict(x_val) # 랜덤포레스트 모델
print(accuracy_score(y_val,p1_2))
print('-'*60)
print(confusion_matrix(y_val, p1_2))
print('-'*60)
print(classification_report(cn[y_val], cn[p1_2]))

p2_1 = m2_1.predict(x_val2) # CNN모델 
p2_2 = p2_1.argmax(axis=1)
print(accuracy_score(y_val,p2_2))
print('-'*60)
print(confusion_matrix(y_val, p2_2))
print('-'*60)
print(classification_report(cn[y_val], cn[p2_2]))

In [ ]:
# 파이프 라인 (Data Pipeline 구성) 함수로 만들기

# 파이프라인에서 필요한 라이브러리/함수
import pandas as pd
import numpy as np
import cv2
from keras.models import load_model

def sign_pipeline(file) :

    # class names 준비
    class_names = list(string.ascii_lowercase)
    class_names = np.array(class_names)

    # 흑백으로 읽기
    img = cv2.imread(file , cv2.IMREAD_GRAYSCALE)

    # 크기 조정
    img = cv2.resize(img, (28, 28))

    # input shape 맞추기
    test_sign = img.reshape(1,28,28,1)

    # 스케일링
    test_sign = test_sign / 255.

    # 모델 로딩
    model = load_model('sign_model.h5')

    # 예측
    pred = model.predict(test_sign)
    pred_1 = pred.argmax(axis=1)

    return class_names[pred_1]

In [ ]:
file = '/content/drive/MyDrive/dataset/test image/v.png'
sign_pipeline(file) 